# OPTION 1: Predicting Diabetes 

Per [the Mayo Clinic](https://www.mayoclinic.org/diseases-conditions/diabetes/diagnosis-treatment/drc-20371451#:~:text=A%20fasting%20blood%20sugar%20level,separate%20tests%2C%20you%20have%20diabetes.), the Glycated Hemoglobin (A1C) test indicates a person's average blood sugar level for the past two to three months. It measures the percentage of blood sugar attached to hemoglobin, the oxygen-carrying protein in red blood cells. This test is a blood test which does not require fasting.

The higher your blood sugar levels, the more hemoglobin you'll have with sugar attached. 
* A1C level of 6.5% or higher on two separate tests indicates that you have diabetes
* A1C between 5.7 and 6.4 % indicates prediabetes
* A1C below 5.7 is considered normal

# OPTION 2: Predicting Insurance Coverage

# Divider

In [428]:
import pandas as pd
pd.set_option('display.max_columns', 0)
import numpy as np

# styling notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

## Data Source

### National Health and Nutrition Examination Survey (NHANES)

The [National Health and Nutrition Examination Survey (NHANES)](https://www.cdc.gov/nchs/nhanes/about_nhanes.htm) is a program of continuous studies designed to assess the health and nutritional status of adults and children in the United States. The survey examines a nationally representative sample of about 5,000 persons located across the country each year. The survey is unique in that it combines interviews and physical examinations. The NHANES interview includes demographic, socioeconomic, dietary, and health-related questions. The examination component consists of medical, dental, and physiological measurements, as well as laboratory tests administered by highly trained medical personnel.

NHANES is a major program of the National Center for Health Statistics (NCHS). NCHS is part of the Centers for Disease Control and Prevention (CDC) and has the responsibility for producing vital and health statistics for the Nation.

### Subset for Predicting Diabetes

While NHANES collects a wealth of demographic, laboratory, and medical data, this analysis and predictive model uses a subset comprised of:

* Demographics - collected by trained interviewers using Computer-Assisted Personal Interview (CAPI) system in either English or Spanish, sometimes with assistance from an interpreter. Individuals 16 years and older and emancipated minors were interviewed directly; a proxy provided information for survey participants who were under 16 and for participants who could not answer the questions themselves.

* Body Measures - measured by trained health technicians in the Mobile Examination Center (MEC). 

* Smoking Survey

* Physical Activity Survey

* Blood Pressure

* Total Cholesterol

* A1C

* Insulin


In [429]:
demographic = pd.read_sas('./data/NHANES2017-2018_demographic.xpt')
insurance = pd.read_sas('./data/NHANES2017-2018_insurance.xpt')
measures = pd.read_sas('./data/NHANES2017-2018_body_measures.xpt')

bp = pd.read_sas('./data/NHANES2017-2018_blood_pressure_oscillometric.xpt')
a1c = pd.read_sas('./data/NHANES2017-2018_a1c.xpt')
chol_total = pd.read_sas('./data/NHANES2017-2018_total_cholesterol.xpt')
chol_hdl = pd.read_sas('./data/NHANES2017-2018_hdl_cholesterol.xpt')
chol_ldl = pd.read_sas('./data/NHANES2017-2018_ldl_cholesterol.xpt')
insulin = pd.read_sas('./data/NHANES2017-2018_insulin.xpt')

activity = pd.read_sas('./data/NHANES2017-2018_physical_activity.xpt')
smoking = pd.read_sas('./data/NHANES2017-2018_smoking.xpt')




In [430]:
data = [demographic, insurance, measures, bp, a1c, chol_total, chol_hdl, chol_ldl, 
        insulin, activity, smoking]

In [431]:
for f in data:
    f.SEQN = f.SEQN.map(lambda x: int(x))
    f.set_index('SEQN', inplace=True)
    display(f.info())
    print('_____'*20)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9254 entries, 93703 to 102956
Data columns (total 45 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SDDSRVYR  9254 non-null   float64
 1   RIDSTATR  9254 non-null   float64
 2   RIAGENDR  9254 non-null   float64
 3   RIDAGEYR  9254 non-null   float64
 4   RIDAGEMN  597 non-null    float64
 5   RIDRETH1  9254 non-null   float64
 6   RIDRETH3  9254 non-null   float64
 7   RIDEXMON  8704 non-null   float64
 8   RIDEXAGM  3433 non-null   float64
 9   DMQMILIZ  6004 non-null   float64
 10  DMQADFC   561 non-null    float64
 11  DMDBORN4  9254 non-null   float64
 12  DMDCITZN  9251 non-null   float64
 13  DMDYRSUS  1948 non-null   float64
 14  DMDEDUC3  2306 non-null   float64
 15  DMDEDUC2  5569 non-null   float64
 16  DMDMARTL  5569 non-null   float64
 17  RIDEXPRG  1110 non-null   float64
 18  SIALANG   9254 non-null   float64
 19  SIAPROXY  9254 non-null   float64
 20  SIAINTRP  9254 non-null 

None

____________________________________________________________________________________________________
<class 'pandas.core.frame.DataFrame'>
Int64Index: 9254 entries, 93703 to 102956
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   HIQ011    9254 non-null   float64
 1   HIQ031A   4254 non-null   float64
 2   HIQ031B   1582 non-null   float64
 3   HIQ031C   58 non-null     float64
 4   HIQ031D   2527 non-null   float64
 5   HIQ031E   93 non-null     float64
 6   HIQ031F   295 non-null    float64
 7   HIQ031H   533 non-null    float64
 8   HIQ031I   301 non-null    float64
 9   HIQ031J   708 non-null    float64
 10  HIQ031AA  1 non-null      float64
 11  HIQ260    172 non-null    float64
 12  HIQ105    1141 non-null   float64
 13  HIQ270    8171 non-null   float64
 14  HIQ210    8171 non-null   float64
dtypes: float64(15)
memory usage: 1.1 MB


None

____________________________________________________________________________________________________
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8704 entries, 93703 to 102956
Data columns (total 20 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   BMDSTATS  8704 non-null   float64
 1   BMXWT     8580 non-null   float64
 2   BMIWT     416 non-null    float64
 3   BMXRECUM  894 non-null    float64
 4   BMIRECUM  24 non-null     float64
 5   BMXHEAD   194 non-null    float64
 6   BMIHEAD   0 non-null      float64
 7   BMXHT     8016 non-null   float64
 8   BMIHT     99 non-null     float64
 9   BMXBMI    8005 non-null   float64
 10  BMXLEG    6703 non-null   float64
 11  BMILEG    334 non-null    float64
 12  BMXARML   8177 non-null   float64
 13  BMIARML   347 non-null    float64
 14  BMXARMC   8173 non-null   float64
 15  BMIARMC   350 non-null    float64
 16  BMXWAIST  7601 non-null   float64
 17  BMIWAIST  437 non-null    float64
 18  B

...

____________________________________________________________________________________________________
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5856 entries, 93705 to 102956
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PAQ605  5856 non-null   float64
 1   PAQ610  1389 non-null   float64
 2   PAD615  1381 non-null   float64
 3   PAQ620  5856 non-null   float64
 4   PAQ625  2439 non-null   float64
 5   PAD630  2426 non-null   float64
 6   PAQ635  5856 non-null   float64
 7   PAQ640  1439 non-null   float64
 8   PAD645  1430 non-null   float64
 9   PAQ650  5856 non-null   float64
 10  PAQ655  1434 non-null   float64
 11  PAD660  1431 non-null   float64
 12  PAQ665  5856 non-null   float64
 13  PAQ670  2308 non-null   float64
 14  PAD675  2301 non-null   float64
 15  PAD680  5846 non-null   float64
dtypes: float64(16)
memory usage: 777.8 KB


None

____________________________________________________________________________________________________
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6724 entries, 93705 to 102956
Data columns (total 36 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SMQ020    5856 non-null   float64
 1   SMD030    2359 non-null   float64
 2   SMQ040    2359 non-null   float64
 3   SMQ050Q   1338 non-null   float64
 4   SMQ050U   1255 non-null   float64
 5   SMD057    1338 non-null   float64
 6   SMQ078    793 non-null    float64
 7   SMD641    1063 non-null   float64
 8   SMD650    1022 non-null   float64
 9   SMD093    1021 non-null   float64
 10  SMDUPCA   6724 non-null   object 
 11  SMD100BR  6724 non-null   object 
 12  SMD100FL  929 non-null    float64
 13  SMD100MN  929 non-null    float64
 14  SMD100LN  929 non-null    float64
 15  SMD100TR  695 non-null    float64
 16  SMD100NI  695 non-null    float64
 17  SMD100CO  695 non-null    float64
 18  S

None

____________________________________________________________________________________________________


#### Demographics 

https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/DEMO_J.htm

Comprised of individual, family, and household-level information. 

In [432]:
demographic.describe().round(1)

,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,RIDEXAGM,DMQMILIZ,DMQADFC,DMDBORN4,DMDCITZN,DMDYRSUS,DMDEDUC3,DMDEDUC2,DMDMARTL,RIDEXPRG,SIALANG,SIAPROXY,SIAINTRP,FIALANG,FIAPROXY,FIAINTRP,MIALANG,MIAPROXY,MIAINTRP,AIALANGA,DMDHHSIZ,DMDFMSIZ,DMDHHSZA,DMDHHSZB,DMDHHSZE,DMDHRGND,DMDHRAGZ,DMDHREDZ,DMDHRMAZ,DMDHSEDZ,WTINT2YR,WTMEC2YR,SDMVPSU,SDMVSTRA,INDHHIN2,INDFMIN2,INDFMPIR
count,9254.0,9254.0,9254.0,9254.0,597.0,9254.0,9254.0,8704.0,3433.0,6004.0,561.0,9254.0,9251.0,1948.0,2306.0,5569.0,5569.0,1110.0,9254.0,9254.0,9254.0,8780.0,8780.0,8780.0,6684.0,6684.0,6684.0,4977.0,9254.0,9254.0,9254.0,9254.0,9254.0,9254.0,9254.0,8764.0,9063.0,4751.0,9254.0,9254.0,9254.0,9254.0,8763.0,8780.0,8023.0
mean,10.0,1.9,1.5,34.3,10.4,3.2,3.5,1.5,107.5,1.9,1.5,1.2,1.1,9.3,6.3,3.5,2.7,2.0,1.1,1.7,2.0,1.1,2.0,2.0,1.1,2.0,2.0,1.1,3.7,3.6,0.5,0.9,0.5,1.5,2.9,2.1,1.5,2.1,34670.7,34670.7,1.5,141.0,12.5,12.2,2.4
std,0.0,0.2,0.5,25.5,7.1,1.3,1.7,0.5,70.6,0.3,0.6,1.6,0.5,18.6,5.8,1.2,3.1,0.4,0.3,0.5,0.2,0.3,0.0,0.2,0.3,0.1,0.1,0.4,1.7,1.8,0.8,1.1,0.8,0.5,0.8,0.7,0.7,0.7,41356.7,43344.0,0.5,4.2,17.3,17.2,1.6
min,10.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,2571.1,0.0,1.0,134.0,1.0,1.0,0.0
25%,10.0,2.0,1.0,11.0,4.0,3.0,3.0,1.0,43.0,2.0,1.0,1.0,1.0,3.0,3.0,3.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,0.0,0.0,0.0,1.0,2.0,2.0,1.0,2.0,13074.4,12347.3,1.0,137.0,6.0,6.0,1.0
50%,10.0,2.0,2.0,31.0,10.0,3.0,3.0,2.0,106.0,2.0,1.0,1.0,1.0,6.0,6.0,4.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,4.0,4.0,0.0,0.0,0.0,1.0,3.0,2.0,1.0,2.0,21098.5,21059.9,2.0,141.0,8.0,8.0,1.9
75%,10.0,2.0,2.0,58.0,17.0,4.0,4.0,2.0,166.0,2.0,2.0,1.0,1.0,7.0,9.0,4.0,5.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,5.0,5.0,1.0,2.0,1.0,2.0,4.0,2.0,2.0,3.0,36923.3,37562.0,2.0,145.0,14.0,14.0,3.7
max,10.0,2.0,2.0,80.0,24.0,5.0,7.0,2.0,239.0,9.0,7.0,99.0,9.0,99.0,66.0,9.0,77.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,7.0,7.0,3.0,3.0,3.0,2.0,4.0,3.0,3.0,3.0,433085.0,419762.8,2.0,148.0,99.0,99.0,5.0


In [433]:
demographic.columns

Index(['SDDSRVYR', 'RIDSTATR', 'RIAGENDR', 'RIDAGEYR', 'RIDAGEMN', 'RIDRETH1',
       'RIDRETH3', 'RIDEXMON', 'RIDEXAGM', 'DMQMILIZ', 'DMQADFC', 'DMDBORN4',
       'DMDCITZN', 'DMDYRSUS', 'DMDEDUC3', 'DMDEDUC2', 'DMDMARTL', 'RIDEXPRG',
       'SIALANG', 'SIAPROXY', 'SIAINTRP', 'FIALANG', 'FIAPROXY', 'FIAINTRP',
       'MIALANG', 'MIAPROXY', 'MIAINTRP', 'AIALANGA', 'DMDHHSIZ', 'DMDFMSIZ',
       'DMDHHSZA', 'DMDHHSZB', 'DMDHHSZE', 'DMDHRGND', 'DMDHRAGZ', 'DMDHREDZ',
       'DMDHRMAZ', 'DMDHSEDZ', 'WTINT2YR', 'WTMEC2YR', 'SDMVPSU', 'SDMVSTRA',
       'INDHHIN2', 'INDFMIN2', 'INDFMPIR'],
      dtype='object')

In [434]:
keepcols_demographic = ['RIAGENDR', 'RIDAGEYR', 'RIDRETH3', 'DMQMILIZ', 'DMDBORN4', 'DMDCITZN',
                    'DMDEDUC2', 'DMDMARTL', 'RIDEXPRG', 'DMDHHSIZ', 'INDHHIN2', 'INDFMIN2', 'INDFMPIR']

In [435]:
keep_demographic = demographic[keepcols_demographic]
keep_demographic.rename(mapper={'RIAGENDR': 'gender',
                                'RIDAGEYR': 'age',
                                'RIDRETH3': 'race',
                                'DMQMILIZ': 'veteran_status',
                                'DMDBORN4': 'country_of_birth',
                                'DMDCITZN': 'citizen_status',
                                'DMDEDUC2': 'education',
                                'DMDMARTL': 'marital_status',
                                'RIDEXPRG': 'pregnancy_status',
                                'INDHHIN2': 'annual_household_income',
                                'INDFMIN2': 'annual_family_income',
                                'INDFMPIR': 'income_poverty_ratio',
                                'DMDHHSIZ': 'household_size'}, 
                       axis=1, inplace=True)
keep_demographic

/Users/kristincooper/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,gender,age,race,veteran_status,country_of_birth,citizen_status,education,marital_status,pregnancy_status,household_size,annual_household_income,annual_family_income,income_poverty_ratio
SEQN,,,,,,,,,,,,,
93703,2.0,2.0,6.0,NaN,1.0,1.0,NaN,NaN,NaN,5.0,15.0,15.0,5.00
93704,1.0,2.0,3.0,NaN,1.0,1.0,NaN,NaN,NaN,4.0,15.0,15.0,5.00
93705,2.0,66.0,4.0,2.0,1.0,1.0,2.0,3.0,NaN,1.0,3.0,3.0,0.82
93706,1.0,18.0,6.0,2.0,1.0,1.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN
93707,1.0,13.0,7.0,NaN,1.0,1.0,NaN,NaN,NaN,7.0,10.0,10.0,1.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...
102952,2.0,70.0,6.0,2.0,2.0,1.0,3.0,1.0,NaN,2.0,4.0,4.0,0.95
102953,1.0,42.0,1.0,2.0,2.0,2.0,3.0,4.0,NaN,1.0,12.0,12.0,NaN
102954,2.0,41.0,4.0,2.0,1.0,1.0,5.0,5.0,2.0,7.0,10.0,10.0,1.18


In [436]:
keep_demographic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9254 entries, 93703 to 102956
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   gender                   9254 non-null   float64
 1   age                      9254 non-null   float64
 2   race                     9254 non-null   float64
 3   veteran_status           6004 non-null   float64
 4   country_of_birth         9254 non-null   float64
 5   citizen_status           9251 non-null   float64
 6   education                5569 non-null   float64
 7   marital_status           5569 non-null   float64
 8   pregnancy_status         1110 non-null   float64
 9   household_size           9254 non-null   float64
 10  annual_household_income  8763 non-null   float64
 11  annual_family_income     8780 non-null   float64
 12  income_poverty_ratio     8023 non-null   float64
dtypes: float64(13)
memory usage: 1012.2 KB


Notes:
* Age 
    * Individuals aged 80 and over are topcoded at 80. In NHANES 2017-2018, the weighted mean age for participants 80+ is 85
    * Individuals' ages are reported in months for those 24 months (2 yrs) and younger
* Income-Poverty Ratio - calculated by dividing family (or individual) income by the poverty guidelines specific to the survey year. The value was not computed if the respondent only reported income as < $20,000 or ≥ $20,000. If family income was reported as a more detailed category, the midpoint of the range was used to compute the ratio. Values at or above 5.00 were coded as 5.00 or more because of disclosure concerns. The values were not computed if the income data was missing.


#### Health Insurance

[Source](https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/HIQ_J.htm)

The Health Insurance questionnaire (variable name prefix HIQ) provides respondent-level interview data on insurance coverage, type of insurance coverage, coverage of prescription drugs, and uninsured status during the past 12 months.

In [437]:
insurance

,HIQ011,HIQ031A,HIQ031B,HIQ031C,HIQ031D,HIQ031E,HIQ031F,HIQ031H,HIQ031I,HIQ031J,HIQ031AA,HIQ260,HIQ105,HIQ270,HIQ210
SEQN,,,,,,,,,,,,,,,
93703,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
93704,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
93705,1.0,NaN,15.0,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,2.0
93706,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
93707,1.0,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102952,1.0,14.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,2.0
102953,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
102954,1.0,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0


In [438]:
insurance.columns

Index(['HIQ011', 'HIQ031A', 'HIQ031B', 'HIQ031C', 'HIQ031D', 'HIQ031E',
       'HIQ031F', 'HIQ031H', 'HIQ031I', 'HIQ031J', 'HIQ031AA', 'HIQ260',
       'HIQ105', 'HIQ270', 'HIQ210'],
      dtype='object')

In [439]:
keepcols_insurance = ['HIQ011', 'HIQ031A', 'HIQ031B', 'HIQ031C', 'HIQ031D', 'HIQ031E',
       'HIQ031F', 'HIQ031H', 'HIQ031I', 'HIQ031J', 'HIQ031AA', 'HIQ270', 'HIQ210']

mapper_insurance = {'HIQ011': 'coverage_status',
          'HIQ031A': 'covered_private',
          'HIQ031B': 'covered_medicare',
          'HIQ031C': 'covered_medigap',
          'HIQ031D': 'covered_medicaid',
          'HIQ031E': 'covered_chip',
          'HIQ031F': 'covered_military',
          'HIQ031H': 'covered_state',
          'HIQ031I': 'covered_other_gov',
          'HIQ031J': 'covered_single_service',
          'HIQ031AA': 'not_covered',
          'HIQ270': 'prescription_coverage',
          'HIQ210': 'uninsured_in_last_year'}

In [440]:
keep_insurance = insurance[keepcols_insurance]
keep_insurance.rename(mapper=mapper_insurance, axis=1, inplace=True)
keep_insurance

/Users/kristincooper/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,coverage_status,covered_private,covered_medicare,covered_medigap,covered_medicaid,covered_chip,covered_military,covered_state,covered_other_gov,covered_single_service,not_covered,prescription_coverage,uninsured_in_last_year
SEQN,,,,,,,,,,,,,
93703,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
93704,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
93705,1.0,NaN,15.0,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
93706,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
93707,1.0,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
102952,1.0,14.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
102953,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
102954,1.0,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0


In [441]:
display(keep_insurance.coverage_status.value_counts())
display(keep_insurance.coverage_status.value_counts(normalize=True))

1.0    8157
2.0    1072
9.0      18
7.0       7
Name: coverage_status, dtype: int64

1.0    0.881457
2.0    0.115842
9.0    0.001945
7.0    0.000756
Name: coverage_status, dtype: float64

In [442]:
display(keep_insurance.uninsured_in_last_year.value_counts())
display(keep_insurance.uninsured_in_last_year.value_counts(normalize=True))

2.0    7591
1.0     561
9.0      18
7.0       1
Name: uninsured_in_last_year, dtype: int64

2.0    0.929017
1.0    0.068657
9.0    0.002203
7.0    0.000122
Name: uninsured_in_last_year, dtype: float64

Coverage status values:
* 1.0 - Yes
* 2.0 - No
* 7.0 - Refused
* 9.0 - Don't know

#### Body Measures

https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/BMX_J.htm

NHANES body measures data are used to monitor trends in infant and child growth, to estimate the prevalence of overweight and obesity in U.S. children, adolescents, and adults, and to examine the associations between body weight and the health and nutritional status of the U.S. population.

All survey participants were eligible for the body measures component. There were no medical, safety, or other exclusions for the body measurements protocol. The health technicians used their discretion to obtain as many measures as practical for persons who used a wheelchair.

In [443]:
measures

,BMDSTATS,BMXWT,BMIWT,BMXRECUM,BMIRECUM,BMXHEAD,BMIHEAD,BMXHT,BMIHT,BMXBMI,BMXLEG,BMILEG,BMXARML,BMIARML,BMXARMC,BMIARMC,BMXWAIST,BMIWAIST,BMXHIP,BMIHIP
SEQN,,,,,,,,,,,,,,,,,,,,
93703,1.0,13.7,3.0,89.6,NaN,NaN,NaN,88.6,NaN,17.5,NaN,NaN,18.0,NaN,16.2,NaN,48.2,NaN,NaN,NaN
93704,1.0,13.9,NaN,95.0,NaN,NaN,NaN,94.2,NaN,15.7,NaN,NaN,18.6,NaN,15.2,NaN,50.0,NaN,NaN,NaN
93705,1.0,79.5,NaN,NaN,NaN,NaN,NaN,158.3,NaN,31.7,37.0,NaN,36.0,NaN,32.0,NaN,101.8,NaN,110.0,NaN
93706,1.0,66.3,NaN,NaN,NaN,NaN,NaN,175.7,NaN,21.5,46.6,NaN,38.8,NaN,27.0,NaN,79.3,NaN,94.4,NaN
93707,1.0,45.4,NaN,NaN,NaN,NaN,NaN,158.4,NaN,18.1,38.1,NaN,33.8,NaN,21.5,NaN,64.1,NaN,83.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102952,1.0,49.0,NaN,NaN,NaN,NaN,NaN,156.5,NaN,20.0,34.4,NaN,32.6,NaN,25.1,NaN,82.2,NaN,87.3,NaN
102953,1.0,97.4,NaN,NaN,NaN,NaN,NaN,164.9,NaN,35.8,38.2,NaN,36.6,NaN,40.6,NaN,114.8,NaN,112.8,NaN
102954,1.0,69.1,NaN,NaN,NaN,NaN,NaN,162.6,NaN,26.1,39.2,NaN,35.2,NaN,26.8,NaN,86.4,NaN,102.7,NaN


In [444]:
measures.columns

Index(['BMDSTATS', 'BMXWT', 'BMIWT', 'BMXRECUM', 'BMIRECUM', 'BMXHEAD',
       'BMIHEAD', 'BMXHT', 'BMIHT', 'BMXBMI', 'BMXLEG', 'BMILEG', 'BMXARML',
       'BMIARML', 'BMXARMC', 'BMIARMC', 'BMXWAIST', 'BMIWAIST', 'BMXHIP',
       'BMIHIP'],
      dtype='object')

In [445]:
keepcols_measures = ['BMXWT', 'BMXHT', 'BMXBMI', 'BMXWAIST', 'BMXHIP']

In [446]:
keep_measures = measures[keepcols_measures]
keep_measures.rename(mapper={'BMXWT': 'weight_kg', 
                             'BMXHT': 'height_cm',
                             'BMXBMI': 'BMI',
                             'BMXWAIST': 'waist_circumference_cm',
                             'BMXHIP': 'hip_circumference_cm'},
                    axis=1, inplace=True)
keep_measures

/Users/kristincooper/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,weight_kg,height_cm,BMI,waist_circumference_cm,hip_circumference_cm
SEQN,,,,,
93703,13.7,88.6,17.5,48.2,NaN
93704,13.9,94.2,15.7,50.0,NaN
93705,79.5,158.3,31.7,101.8,110.0
93706,66.3,175.7,21.5,79.3,94.4
93707,45.4,158.4,18.1,64.1,83.0
...,...,...,...,...,...
102952,49.0,156.5,20.0,82.2,87.3
102953,97.4,164.9,35.8,114.8,112.8
102954,69.1,162.6,26.1,86.4,102.7


#### Blood Pressure

https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/BPXO_J.htm

Blood pressure measures used in this analysis were taken using a digital upper-arm electronic blood pressure measurement device. 3 measurements were taken 60 seconds apart after the individual had been resting quietly in a seated position for 5 minutes. For the purposes of this analysis, I have calculated the average systolic and average diastolic measurements across the 3 readings.

In [447]:
bp

,BPAOARM,BPAOCSZ,BPAOMNTS,BPXOSY1,BPXODI1,BPXOSY2,BPXODI2,BPXOSY3,BPXODI3,BPXOPLS1,BPXOPLS2,BPXOPLS3
SEQN,,,,,,,,,,,,
93705,b'R',4.0,-20.0,164.0,66.0,165.0,66.0,172.0,66.0,52.0,51.0,49.0
93706,b'R',3.0,138.0,126.0,74.0,128.0,68.0,133.0,71.0,76.0,83.0,73.0
93707,b'R',2.0,12.0,136.0,71.0,133.0,72.0,139.0,71.0,100.0,89.0,91.0
93708,b'R',3.0,22.0,146.0,82.0,142.0,76.0,151.0,81.0,67.0,65.0,71.0
93709,b'R',4.0,58.0,120.0,83.0,124.0,81.0,113.0,74.0,64.0,62.0,61.0
...,...,...,...,...,...,...,...,...,...,...,...,...
102952,b'R',3.0,97.0,154.0,92.0,144.0,84.0,145.0,69.0,88.0,84.0,74.0
102953,b'R',4.0,-57.0,135.0,91.0,133.0,86.0,132.0,83.0,76.0,79.0,78.0
102954,b'',3.0,-101.0,123.0,75.0,119.0,71.0,122.0,73.0,NaN,NaN,NaN


In [448]:
bp.columns

Index(['BPAOARM', 'BPAOCSZ', 'BPAOMNTS', 'BPXOSY1', 'BPXODI1', 'BPXOSY2',
       'BPXODI2', 'BPXOSY3', 'BPXODI3', 'BPXOPLS1', 'BPXOPLS2', 'BPXOPLS3'],
      dtype='object')

In [449]:
keepcols_bp = ['BPXOSY1', 'BPXODI1', 'BPXOSY2', 'BPXODI2', 'BPXOSY3', 
               'BPXODI3', 'BPXOPLS1', 'BPXOPLS2', 'BPXOPLS3']

mapper_bp = {'BPXOSY1': 'sys_1', 
          'BPXODI1': 'dias_1',
          'BPXOSY2': 'sys_2',
          'BPXODI2': 'dias_2',
          'BPXOSY3': 'sys_3',
          'BPXODI3': 'dias_3',
          'BPXOPLS1': 'pulse_1',
          'BPXOPLS2': 'pulse_2',
          'BPXOPLS3': 'pulse_3'}

In [450]:
keep_bp = bp[keepcols_bp]
keep_bp.rename(mapper=mapper_bp, axis=1, inplace=True)
keep_bp

/Users/kristincooper/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,sys_1,dias_1,sys_2,dias_2,sys_3,dias_3,pulse_1,pulse_2,pulse_3
SEQN,,,,,,,,,
93705,164.0,66.0,165.0,66.0,172.0,66.0,52.0,51.0,49.0
93706,126.0,74.0,128.0,68.0,133.0,71.0,76.0,83.0,73.0
93707,136.0,71.0,133.0,72.0,139.0,71.0,100.0,89.0,91.0
93708,146.0,82.0,142.0,76.0,151.0,81.0,67.0,65.0,71.0
93709,120.0,83.0,124.0,81.0,113.0,74.0,64.0,62.0,61.0
...,...,...,...,...,...,...,...,...,...
102952,154.0,92.0,144.0,84.0,145.0,69.0,88.0,84.0,74.0
102953,135.0,91.0,133.0,86.0,132.0,83.0,76.0,79.0,78.0
102954,123.0,75.0,119.0,71.0,122.0,73.0,NaN,NaN,NaN


In [451]:
keep_bp['avg_systolic'] = ((keep_bp['sys_1'] + keep_bp['sys_2'] + keep_bp['sys_3']) / 3).round(1)
keep_bp['avg_diastolic'] = ((keep_bp['dias_1'] + keep_bp['dias_2'] + keep_bp['dias_3']) / 3).round(1)
keep_bp['avg_pulse'] = ((keep_bp['pulse_1'] + keep_bp['pulse_2'] + keep_bp['pulse_3']) / 3).round(1)
keep_bp

<ipython-input-451-9a2353af5978>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keep_bp['avg_systolic'] = ((keep_bp['sys_1'] + keep_bp['sys_2'] + keep_bp['sys_3']) / 3).round(1)
<ipython-input-451-9a2353af5978>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keep_bp['avg_diastolic'] = ((keep_bp['dias_1'] + keep_bp['dias_2'] + keep_bp['dias_3']) / 3).round(1)


,sys_1,dias_1,sys_2,dias_2,sys_3,dias_3,pulse_1,pulse_2,pulse_3,avg_systolic,avg_diastolic,avg_pulse
SEQN,,,,,,,,,,,,
93705,164.0,66.0,165.0,66.0,172.0,66.0,52.0,51.0,49.0,167.0,66.0,50.7
93706,126.0,74.0,128.0,68.0,133.0,71.0,76.0,83.0,73.0,129.0,71.0,77.3
93707,136.0,71.0,133.0,72.0,139.0,71.0,100.0,89.0,91.0,136.0,71.3,93.3
93708,146.0,82.0,142.0,76.0,151.0,81.0,67.0,65.0,71.0,146.3,79.7,67.7
93709,120.0,83.0,124.0,81.0,113.0,74.0,64.0,62.0,61.0,119.0,79.3,62.3
...,...,...,...,...,...,...,...,...,...,...,...,...
102952,154.0,92.0,144.0,84.0,145.0,69.0,88.0,84.0,74.0,147.7,81.7,82.0
102953,135.0,91.0,133.0,86.0,132.0,83.0,76.0,79.0,78.0,133.3,86.7,77.7
102954,123.0,75.0,119.0,71.0,122.0,73.0,NaN,NaN,NaN,121.3,73.0,NaN


In [452]:
keep_bp2 = keep_bp[['avg_systolic', 'avg_diastolic', 'avg_pulse']]
keep_bp2

,avg_systolic,avg_diastolic,avg_pulse
SEQN,,,
93705,167.0,66.0,50.7
93706,129.0,71.0,77.3
93707,136.0,71.3,93.3
93708,146.3,79.7,67.7
93709,119.0,79.3,62.3
...,...,...,...
102952,147.7,81.7,82.0
102953,133.3,86.7,77.7
102954,121.3,73.0,NaN


#### Lab - A1C

In [453]:
a1c

,LBXGH
SEQN,
93705,6.2
93706,5.2
93707,5.6
93708,6.2
93709,6.3
...,...
102952,7.4
102953,5.9
102954,5.2


In [454]:
a1c.rename(mapper={'LBXGH': 'glycohemoglobin'}, axis=1, inplace=True)

In [455]:
a1c.isna().sum()

glycohemoglobin    356
dtype: int64

In [456]:
a1c.dropna(inplace=True)

In [457]:
def diagnose(a1c_value):
    diagnosis = None
    if a1c_value >= 6.5:
        diagnosis = 'Diabetic'
    if (a1c_value >= 5.7) & (a1c_value < 6.5):
        diagnosis = 'Prediabetic'
    if a1c_value < 5.7:
        diagnosis = 'Normal'
    return diagnosis

In [458]:
a1c['calculated_diagnosis'] = a1c.glycohemoglobin.map(lambda x: diagnose(x))
a1c

,glycohemoglobin,calculated_diagnosis
SEQN,,
93705,6.2,Prediabetic
93706,5.2,Normal
93707,5.6,Normal
93708,6.2,Prediabetic
93709,6.3,Prediabetic
...,...,...
102952,7.4,Diabetic
102953,5.9,Prediabetic
102954,5.2,Normal


In [459]:
display(a1c.calculated_diagnosis.value_counts())
display(a1c.calculated_diagnosis.value_counts(normalize=True).round(2))

Normal         3659
Prediabetic    1635
Diabetic        751
Name: calculated_diagnosis, dtype: int64

Normal         0.61
Prediabetic    0.27
Diabetic       0.12
Name: calculated_diagnosis, dtype: float64

#### Lab - Cholesterol

https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/TCHOL_J.htm

https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/TRIGLY_J.htm

https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/HDL_J.htm

A complete cholesterol test — also called a lipid panel or lipid profile — is a blood test that can measure the amount of cholesterol and triglycerides in your blood. The blood lipids measurements in NHANES include total cholesterol, high-density lipoprotein cholesterol (HDL-C), low-density lipoproteins cholesterol (LDL-C), and triglycerides. 

* **Total cholesterol** - sum of your blood's cholesterol content
    * Less than 200 mg/dL - desirable
    * 200-239 mg/dL - borderline
    * Greater than 240 mg/dL - high
* **High-density lipoprotein (HDL) cholesterol** - the "good" cholesterol because it helps carry away LDL cholesterol, thus keeping arteries open and your blood flowing more freely.
    * Greater than 60 mg/dL - best
    * 40-59 mg/dL - good
    * Less than 50 (women) or 40 (men) - poor
* **Low-density lipoprotein (LDL) cholesterol** "bad" cholesterol; too much of it in your blood causes the buildup of fatty deposits (plaques) in your arteries (atherosclerosis), which reduces blood flow. These plaques sometimes rupture and can lead to a heart attack or stroke.
    * Less than 100 mg/dL - optimal
    * 100-129 mg/dL - high for those with coronary artery disease 
    * 130-159 mg/dL - borderline
    * Greater than 160 mg/dL - high
    * Greater than 190 mg/dL - very high
* **Triglycerides** - a type of fat in the blood created from calories your body doesn't need. High triglyceride levels are associated with several factors, including being overweight, eating too many sweets or drinking too much alcohol, smoking, being sedentary, or having diabetes with elevated blood sugar levels.
    * Less than 150 mg/dL - desirable
    * 150-199 mg/dL - borderline
    * 200-499 mg/dL - high
    * Greater than 500 mg/dL - very high

https://www.mayoclinic.org/tests-procedures/cholesterol-test/about/pac-20384601

In [460]:
chol_total = chol_total.drop(columns='LBDTCSI').rename(mapper={'LBXTC': 'cholesterol_total'}, axis=1)
chol_total

,cholesterol_total
SEQN,
93705,157.0
93706,148.0
93707,189.0
93708,209.0
93709,176.0
...,...
102952,119.0
102953,182.0
102954,172.0


In [461]:
chol_hdl = chol_hdl.drop(columns='LBDHDDSI').rename(mapper={'LBDHDD': 'cholesterol_hdl'}, axis=1)
chol_hdl

,cholesterol_hdl
SEQN,
93705,60.0
93706,47.0
93707,68.0
93708,88.0
93709,65.0
...,...
102952,60.0
102953,49.0
102954,54.0


In [462]:
chol_ldl

,WTSAF2YR,LBXTR,LBDTRSI,LBDLDL,LBDLDLSI,LBDLDLM,LBDLDMSI,LBDLDLN,LBDLDNSI
SEQN,,,,,,,,,
93708,2.565368e+04,58.0,0.655,109.0,2.819,107.0,2.767,111.0,2.870
93711,2.922605e+04,48.0,0.542,156.0,4.034,152.0,3.931,158.0,4.086
93717,1.371510e+05,102.0,1.152,140.0,3.620,138.0,3.569,142.0,3.672
93718,5.888331e+04,46.0,0.519,80.0,2.069,77.0,1.991,79.0,2.043
93719,5.397605e-79,80.0,0.903,35.0,0.905,35.0,0.905,35.0,0.905
...,...,...,...,...,...,...,...,...,...
102947,3.208847e+04,35.0,0.395,66.0,1.707,62.0,1.603,64.0,1.655
102948,2.911334e+05,52.0,0.587,123.0,3.181,119.0,3.077,124.0,3.207
102952,3.273305e+04,78.0,0.881,43.0,1.112,43.0,1.112,43.0,1.112


In [463]:
chol_ldl.describe().round(1)

,WTSAF2YR,LBXTR,LBDTRSI,LBDLDL,LBDLDLSI,LBDLDLM,LBDLDMSI,LBDLDLN,LBDLDNSI
count,3036.0,2834.0,2834.0,2808.0,2808.0,2808.0,2808.0,2827.0,2827.0
mean,89741.9,107.3,1.2,106.9,2.8,107.2,2.8,108.5,2.8
std,109439.5,98.3,1.1,35.6,0.9,35.6,0.9,36.3,0.9
min,0.0,10.0,0.1,18.0,0.5,21.0,0.5,20.0,0.5
25%,29748.3,58.0,0.7,82.0,2.1,82.0,2.1,83.0,2.1
50%,55057.3,87.0,1.0,103.0,2.7,104.0,2.7,105.0,2.7
75%,101741.7,130.0,1.5,128.0,3.3,129.0,3.3,130.0,3.4
max,944153.2,2684.0,30.3,357.0,9.2,358.0,9.3,359.0,9.3


In [464]:
chol_ldl.columns

Index(['WTSAF2YR', 'LBXTR', 'LBDTRSI', 'LBDLDL', 'LBDLDLSI', 'LBDLDLM',
       'LBDLDMSI', 'LBDLDLN', 'LBDLDNSI'],
      dtype='object')

In [465]:
keepcols_chol_ldl = ['LBXTR', 'LBDLDL']
mapper_chol_ldl = {'LBXTR': 'triglyceride', 'LBDLDL': 'cholesterol_ldl'}

In [466]:
keep_chol_ldl = chol_ldl[keepcols_chol_ldl].rename(mapper=mapper_chol_ldl, axis=1)
keep_chol_ldl

,triglyceride,cholesterol_ldl
SEQN,,
93708,58.0,109.0
93711,48.0,156.0
93717,102.0,140.0
93718,46.0,80.0
93719,80.0,35.0
...,...,...
102947,35.0,66.0
102948,52.0,123.0
102952,78.0,43.0


In [467]:
chol1 = pd.merge(keep_chol_ldl, chol_hdl, how='outer', on='SEQN')
chol1

,triglyceride,cholesterol_ldl,cholesterol_hdl
SEQN,,,
93708,58.0,109.0,88.0
93711,48.0,156.0,72.0
93717,102.0,140.0,53.0
93718,46.0,80.0,63.0
93719,80.0,35.0,46.0
...,...,...,...
102944,NaN,NaN,50.0
102946,NaN,NaN,50.0
102949,NaN,NaN,73.0


In [468]:
cholesterol = pd.merge(chol1, chol_total, how='outer', on='SEQN')
cholesterol

,triglyceride,cholesterol_ldl,cholesterol_hdl,cholesterol_total
SEQN,,,,
93708,58.0,109.0,88.0,209.0
93711,48.0,156.0,72.0,238.0
93717,102.0,140.0,53.0,213.0
93718,46.0,80.0,63.0,152.0
93719,80.0,35.0,46.0,97.0
...,...,...,...,...
102944,NaN,NaN,50.0,214.0
102946,NaN,NaN,50.0,214.0
102949,NaN,NaN,73.0,201.0


#### Lab - Insulin

In [469]:
insulin

,WTSAF2YR,LBXIN,LBDINSI,LBDINLC
SEQN,,,,
93708,2.565368e+04,9.72,58.32,5.397605e-79
93711,2.922605e+04,5.28,31.68,5.397605e-79
93717,1.371510e+05,3.94,23.64,5.397605e-79
93718,5.888331e+04,4.89,29.34,5.397605e-79
93719,5.397605e-79,10.94,65.64,5.397605e-79
...,...,...,...,...
102947,3.208847e+04,0.71,4.26,1.000000e+00
102948,2.911334e+05,5.42,32.52,5.397605e-79
102952,3.273305e+04,4.39,26.34,5.397605e-79


In [470]:
keep_insulin = pd.DataFrame(insulin['LBXIN'])
keep_insulin.rename(mapper={'LBXIN': 'insulin'}, axis=1, inplace=True)

In [471]:
keep_insulin

,insulin
SEQN,
93708,9.72
93711,5.28
93717,3.94
93718,4.89
93719,10.94
...,...
102947,0.71
102948,5.42
102952,4.39


#### Physical Activity

https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/PAQ_J.htm

Data obtained from the respondent-level interview and questionnaire on physical activity.

In [472]:
activity

,PAQ605,PAQ610,PAD615,PAQ620,PAQ625,PAD630,PAQ635,PAQ640,PAD645,PAQ650,PAQ655,PAD660,PAQ665,PAQ670,PAD675,PAD680
SEQN,,,,,,,,,,,,,,,,
93705,2.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,1.0,2.0,60.0,300.0
93706,2.0,NaN,NaN,2.0,NaN,NaN,1.0,5.0,45.0,2.0,NaN,NaN,1.0,2.0,30.0,240.0
93708,2.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,1.0,5.0,30.0,120.0
93709,2.0,NaN,NaN,1.0,2.0,180.0,2.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,600.0
93711,2.0,NaN,NaN,2.0,NaN,NaN,1.0,5.0,60.0,1.0,4.0,60.0,1.0,2.0,30.0,420.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102950,2.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,60.0
102952,2.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,1.0,6.0,60.0,120.0
102953,1.0,3.0,240.0,1.0,3.0,240.0,2.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,360.0


In [473]:
activity.columns

Index(['PAQ605', 'PAQ610', 'PAD615', 'PAQ620', 'PAQ625', 'PAD630', 'PAQ635',
       'PAQ640', 'PAD645', 'PAQ650', 'PAQ655', 'PAD660', 'PAQ665', 'PAQ670',
       'PAD675', 'PAD680'],
      dtype='object')

In [474]:
keepcols_activity = ['PAD615', 'PAQ610', 'PAD630', 'PAQ625', 'PAQ640', 'PAD645', 'PAD660', 
                     'PAQ655', 'PAD675', 'PAQ670', 'PAD680']
mapper_activity = {'PAD615':'work_vigorous_minperday', 
                   'PAQ610': 'work_vigorous_daysperweek',
                   'PAD630': 'work_moderate_minperday', 
                   'PAQ625': 'work_moderate_daysperweek',
                   'PAD645': 'transportation_minperday', 
                   'PAQ640': 'transportation_daysperweek',
                   'PAD660': 'recreation_vigorous_minperday', 
                   'PAQ655': 'recreation_vigorous_daysperweek',
                   'PAD675': 'recreation_moderate_minperday', 
                   'PAQ670': 'recreation_moderate_daysperweek',
                   'PAD680': 'sedentary_minsperday'}

In [475]:
keep_activity = activity[keepcols_activity].rename(mapper=mapper_activity, axis=1).fillna(0)
keep_activity

,work_vigorous_minperday,work_vigorous_daysperweek,work_moderate_minperday,work_moderate_daysperweek,transportation_daysperweek,transportation_minperday,recreation_vigorous_minperday,recreation_vigorous_daysperweek,recreation_moderate_minperday,recreation_moderate_daysperweek,sedentary_minsperday
SEQN,,,,,,,,,,,
93705,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,2.0,300.0
93706,0.0,0.0,0.0,0.0,5.0,45.0,0.0,0.0,30.0,2.0,240.0
93708,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,5.0,120.0
93709,0.0,0.0,180.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,600.0
93711,0.0,0.0,0.0,0.0,5.0,60.0,60.0,4.0,30.0,2.0,420.0
...,...,...,...,...,...,...,...,...,...,...,...
102950,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0
102952,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,6.0,120.0
102953,240.0,3.0,240.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,360.0


In [476]:
days_cols = ['work_vigorous_daysperweek', 'work_moderate_daysperweek', 'transportation_daysperweek',
            'recreation_vigorous_daysperweek', 'recreation_moderate_daysperweek']
mins_cols = ['work_vigorous_minperday', 'work_moderate_minperday', 'transportation_minperday',
            'recreation_vigorous_minperday', 'recreation_moderate_minperday', ]

In [477]:
for col in days_cols:
    print(col)
    display(keep_activity[col].value_counts())
    print('_____'*20)

work_vigorous_daysperweek


0.0     4467
5.0      384
3.0      232
2.0      178
6.0      160
4.0      153
7.0      151
1.0      130
99.0       1
Name: work_vigorous_daysperweek, dtype: int64

____________________________________________________________________________________________________
work_moderate_daysperweek


0.0     3417
5.0      702
3.0      401
7.0      331
2.0      308
4.0      307
6.0      210
1.0      174
99.0       6
Name: work_moderate_daysperweek, dtype: int64

____________________________________________________________________________________________________
transportation_daysperweek


...

____________________________________________________________________________________________________
recreation_vigorous_daysperweek


0.0     4422
3.0      385
2.0      269
4.0      264
5.0      207
1.0      163
7.0       73
6.0       72
99.0       1
Name: recreation_vigorous_daysperweek, dtype: int64

____________________________________________________________________________________________________
recreation_moderate_daysperweek


0.0     3548
3.0      606
2.0      491
5.0      321
4.0      293
1.0      253
7.0      250
6.0       91
99.0       3
Name: recreation_moderate_daysperweek, dtype: int64

____________________________________________________________________________________________________


In [478]:
for col in mins_cols:
    print(col)
    display(keep_activity[col].value_counts())
    print('_____'*20)

work_vigorous_minperday


0.0       4475
120.0      234
60.0       194
240.0      177
180.0      130
480.0      107
30.0       106
300.0       93
360.0       80
420.0       41
10.0        40
600.0       38
15.0        31
20.0        31
90.0        23
540.0       15
45.0        14
9999.0       6
150.0        4
40.0         4
720.0        4
25.0         2
80.0         1
12.0         1
840.0        1
35.0         1
780.0        1
140.0        1
660.0        1
Name: work_vigorous_minperday, dtype: int64

____________________________________________________________________________________________________
work_moderate_minperday


0.0       3430
120.0      449
60.0       362
240.0      317
180.0      277
30.0       206
300.0      150
480.0      131
360.0      125
10.0        76
20.0        75
15.0        55
90.0        39
420.0       38
600.0       34
45.0        28
540.0       14
9999.0      13
40.0        11
720.0        6
150.0        6
25.0         4
50.0         2
660.0        1
12.0         1
230.0        1
70.0         1
35.0         1
840.0        1
140.0        1
210.0        1
Name: work_moderate_minperday, dtype: int64

____________________________________________________________________________________________________
transportation_minperday


...

____________________________________________________________________________________________________
recreation_vigorous_minperday


0.0       4425
60.0       464
120.0      240
30.0       210
45.0       110
90.0       110
180.0       90
20.0        62
40.0        29
240.0       23
15.0        18
10.0        13
25.0        11
35.0        10
150.0       10
50.0         5
75.0         5
80.0         4
300.0        4
18.0         2
480.0        2
360.0        2
23.0         1
12.0         1
11.0         1
209.0        1
420.0        1
55.0         1
9999.0       1
Name: recreation_vigorous_minperday, dtype: int64

____________________________________________________________________________________________________
recreation_moderate_minperday


0.0       3555
60.0       641
30.0       557
120.0      260
20.0       194
45.0       123
15.0       102
90.0        80
180.0       77
10.0        73
40.0        59
240.0       54
300.0       16
25.0        16
35.0         8
50.0         7
360.0        6
80.0         4
75.0         4
480.0        4
12.0         3
22.0         2
420.0        2
150.0        2
540.0        1
55.0         1
16.0         1
28.0         1
100.0        1
210.0        1
9999.0       1
Name: recreation_moderate_minperday, dtype: int64

____________________________________________________________________________________________________


In [479]:
for col in days_cols:
    keep_activity[col].replace({77.0: 0, 99.0:0}, inplace=True)
for col in mins_cols:
    keep_activity[col].replace({7777.0: 0, 9999.0: 0}, inplace=True)

In [480]:
for col in days_cols:
    print(col)
    display(keep_activity[col].value_counts())
    print('_____'*20)

work_vigorous_daysperweek


0.0    4468
5.0     384
3.0     232
2.0     178
6.0     160
4.0     153
7.0     151
1.0     130
Name: work_vigorous_daysperweek, dtype: int64

____________________________________________________________________________________________________
work_moderate_daysperweek


0.0    3423
5.0     702
3.0     401
7.0     331
2.0     308
4.0     307
6.0     210
1.0     174
Name: work_moderate_daysperweek, dtype: int64

____________________________________________________________________________________________________
transportation_daysperweek


...

____________________________________________________________________________________________________
recreation_vigorous_daysperweek


0.0    4423
3.0     385
2.0     269
4.0     264
5.0     207
1.0     163
7.0      73
6.0      72
Name: recreation_vigorous_daysperweek, dtype: int64

____________________________________________________________________________________________________
recreation_moderate_daysperweek


0.0    3551
3.0     606
2.0     491
5.0     321
4.0     293
1.0     253
7.0     250
6.0      91
Name: recreation_moderate_daysperweek, dtype: int64

____________________________________________________________________________________________________


In [481]:
for col in mins_cols:
    print(col)
    display(keep_activity[col].value_counts())
    print('_____'*20)

work_vigorous_minperday


0.0      4481
120.0     234
60.0      194
240.0     177
180.0     130
480.0     107
30.0      106
300.0      93
360.0      80
420.0      41
10.0       40
600.0      38
20.0       31
15.0       31
90.0       23
540.0      15
45.0       14
150.0       4
40.0        4
720.0       4
25.0        2
80.0        1
12.0        1
840.0       1
35.0        1
780.0       1
140.0       1
660.0       1
Name: work_vigorous_minperday, dtype: int64

____________________________________________________________________________________________________
work_moderate_minperday


0.0      3443
120.0     449
60.0      362
240.0     317
180.0     277
30.0      206
300.0     150
480.0     131
360.0     125
10.0       76
20.0       75
15.0       55
90.0       39
420.0      38
600.0      34
45.0       28
540.0      14
40.0       11
150.0       6
720.0       6
25.0        4
50.0        2
660.0       1
12.0        1
230.0       1
70.0        1
35.0        1
840.0       1
140.0       1
210.0       1
Name: work_moderate_minperday, dtype: int64

____________________________________________________________________________________________________
transportation_minperday


...

____________________________________________________________________________________________________
recreation_vigorous_minperday


0.0      4426
60.0      464
120.0     240
30.0      210
45.0      110
90.0      110
180.0      90
20.0       62
40.0       29
240.0      23
15.0       18
10.0       13
25.0       11
150.0      10
35.0       10
50.0        5
75.0        5
80.0        4
300.0       4
18.0        2
480.0       2
360.0       2
209.0       1
12.0        1
420.0       1
23.0        1
55.0        1
11.0        1
Name: recreation_vigorous_minperday, dtype: int64

____________________________________________________________________________________________________
recreation_moderate_minperday


0.0      3556
60.0      641
30.0      557
120.0     260
20.0      194
45.0      123
15.0      102
90.0       80
180.0      77
10.0       73
40.0       59
240.0      54
300.0      16
25.0       16
35.0        8
50.0        7
360.0       6
80.0        4
75.0        4
480.0       4
12.0        3
420.0       2
150.0       2
22.0        2
540.0       1
55.0        1
16.0        1
28.0        1
100.0       1
210.0       1
Name: recreation_moderate_minperday, dtype: int64

____________________________________________________________________________________________________


In [482]:
keep_activity['vigorous_activity_minsperweek'] =\
            (keep_activity['work_vigorous_daysperweek']*keep_activity['work_vigorous_minperday']) +\
            (keep_activity['recreation_vigorous_daysperweek']*keep_activity['recreation_vigorous_daysperweek'])

keep_activity['moderate_activity_minsperweek'] =\
            (keep_activity['work_moderate_daysperweek']*keep_activity['work_moderate_minperday']) +\
            (keep_activity['recreation_moderate_daysperweek']*keep_activity['recreation_moderate_minperday']) +\
            (keep_activity['transportation_daysperweek']*keep_activity['transportation_minperday'])

keep_activity

,work_vigorous_minperday,work_vigorous_daysperweek,work_moderate_minperday,work_moderate_daysperweek,transportation_daysperweek,transportation_minperday,recreation_vigorous_minperday,recreation_vigorous_daysperweek,recreation_moderate_minperday,recreation_moderate_daysperweek,sedentary_minsperday,vigorous_activity_minsperweek,moderate_activity_minsperweek
SEQN,,,,,,,,,,,,,
93705,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,2.0,300.0,0.0,120.0
93706,0.0,0.0,0.0,0.0,5.0,45.0,0.0,0.0,30.0,2.0,240.0,0.0,285.0
93708,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,5.0,120.0,0.0,150.0
93709,0.0,0.0,180.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,600.0,0.0,360.0
93711,0.0,0.0,0.0,0.0,5.0,60.0,60.0,4.0,30.0,2.0,420.0,16.0,360.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
102950,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,0.0,0.0
102952,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,6.0,120.0,0.0,360.0
102953,240.0,3.0,240.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,360.0,720.0,720.0


In [483]:
keep_activity2 = keep_activity[['sedentary_minsperday',
                                'vigorous_activity_minsperweek',
                                'moderate_activity_minsperweek']]
keep_activity2

,sedentary_minsperday,vigorous_activity_minsperweek,moderate_activity_minsperweek
SEQN,,,
93705,300.0,0.0,120.0
93706,240.0,0.0,285.0
93708,120.0,0.0,150.0
93709,600.0,0.0,360.0
93711,420.0,16.0,360.0
...,...,...,...
102950,60.0,0.0,0.0
102952,120.0,0.0,360.0
102953,360.0,720.0,720.0


In [484]:
keep_activity2.describe().round(2)

,sedentary_minsperday,vigorous_activity_minsperweek,moderate_activity_minsperweek
count,5856.00,5856.00,5856.00
mean,388.89,222.20,491.14
std,771.32,603.94,777.00
min,0.00,0.00,0.00
25%,180.00,0.00,0.00
50%,300.00,0.00,150.00
75%,480.00,25.00,600.00
max,9999.00,5056.00,5880.00


#### Smoking

https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/SMQ_J.htm#Component_Description

History of cigarette use per respondent, self-reported during interviews.

In [485]:
smoking.head()

,SMQ020,SMD030,SMQ040,SMQ050Q,SMQ050U,SMD057,SMQ078,SMD641,SMD650,SMD093,SMDUPCA,SMD100BR,SMD100FL,SMD100MN,SMD100LN,SMD100TR,SMD100NI,SMD100CO,SMQ621,SMD630,SMQ661,SMQ665A,SMQ665B,SMQ665C,SMQ665D,SMQ670,SMQ848,SMQ852Q,SMQ852U,SMQ890,SMQ895,SMQ900,SMQ905,SMQ910,SMQ915,SMAQUEX2
SEQN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
93705,1.0,16.0,3.0,30.0,4.0,5.0,NaN,NaN,NaN,NaN,b'',b'',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.0,NaN,2.0,NaN,1.0
93706,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b'',b'',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.0,NaN,2.0,NaN,1.0
93707,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b'',b'',NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
93708,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b'',b'',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.0,NaN,2.0,NaN,1.0
93709,1.0,15.0,1.0,NaN,NaN,NaN,1.0,30.0,5.0,1.0,b'* NO MATCH *',b'WAVE GREEN',1.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,5.397605e-79,1.0,5.397605e-79,1.0,5.397605e-79,1.0


In [486]:
keepcols_smoking = ['SMQ020', 'SMQ040', 'SMQ905', 'SMQ915']
mapper_smoking = {'SMQ020': 'lifetime_cigarette_smoker', 
                 'SMQ040': 'current_cigarette_smoker',
                 'SMQ905': 'ecig_smoker',
                 'SMQ915': 'smokeless_tobacco_user'}

In [487]:
keep_smoking = smoking[keepcols_smoking].rename(mapper=mapper_smoking, axis=1)
keep_smoking.round(1)

,lifetime_cigarette_smoker,current_cigarette_smoker,ecig_smoker,smokeless_tobacco_user
SEQN,,,,
93705,1.0,3.0,NaN,NaN
93706,2.0,NaN,NaN,NaN
93707,NaN,NaN,NaN,NaN
93708,2.0,NaN,NaN,NaN
93709,1.0,1.0,0.0,0.0
...,...,...,...,...
102952,2.0,NaN,NaN,NaN
102953,1.0,3.0,0.0,0.0
102954,2.0,NaN,NaN,NaN


## Merging and Preprocessing Data

NOTES TO SELF:
* Focus on adults; remove ages below 18 from dataset
* Remove pregnant women from dataset as their measurements may be skewed

### Merge Data

In [488]:
dfs = [a1c, keep_demographic, keep_insurance, keep_measures, keep_bp2, cholesterol, 
       keep_insulin, keep_activity2, keep_smoking]

In [489]:
col_count = 0
for df in dfs:
    col_count += len(list(df.columns))
col_count

48

In [490]:
dfs_preds = dfs[1:]

In [491]:
# Concatenated all data besides my target

df_preds = pd.concat(dfs_preds, join='outer', axis=1)
df_preds

,gender,age,race,veteran_status,country_of_birth,citizen_status,education,marital_status,pregnancy_status,household_size,annual_household_income,annual_family_income,income_poverty_ratio,coverage_status,covered_private,covered_medicare,covered_medigap,covered_medicaid,covered_chip,covered_military,covered_state,covered_other_gov,covered_single_service,not_covered,prescription_coverage,uninsured_in_last_year,weight_kg,height_cm,BMI,waist_circumference_cm,hip_circumference_cm,avg_systolic,avg_diastolic,avg_pulse,triglyceride,cholesterol_ldl,cholesterol_hdl,cholesterol_total,insulin,sedentary_minsperday,vigorous_activity_minsperweek,moderate_activity_minsperweek,lifetime_cigarette_smoker,current_cigarette_smoker,ecig_smoker,smokeless_tobacco_user
SEQN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
93703,2.0,2.0,6.0,NaN,1.0,1.0,NaN,NaN,NaN,5.0,15.0,15.0,5.00,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,13.7,88.6,17.5,48.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93704,1.0,2.0,3.0,NaN,1.0,1.0,NaN,NaN,NaN,4.0,15.0,15.0,5.00,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,13.9,94.2,15.7,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93705,2.0,66.0,4.0,2.0,1.0,1.0,2.0,3.0,NaN,1.0,3.0,3.0,0.82,1.0,NaN,15.0,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,79.5,158.3,31.7,101.8,110.0,167.0,66.0,50.7,NaN,NaN,60.0,157.0,NaN,300.0,0.0,120.0,1.0,3.0,NaN,NaN
93706,1.0,18.0,6.0,2.0,1.0,1.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,66.3,175.7,21.5,79.3,94.4,129.0,71.0,77.3,NaN,NaN,47.0,148.0,NaN,240.0,0.0,285.0,2.0,NaN,NaN,NaN
93707,1.0,13.0,7.0,NaN,1.0,1.0,NaN,NaN,NaN,7.0,10.0,10.0,1.88,1.0,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,45.4,158.4,18.1,64.1,83.0,136.0,71.3,93.3,NaN,NaN,68.0,189.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102952,2.0,70.0,6.0,2.0,2.0,1.0,3.0,1.0,NaN,2.0,4.0,4.0,0.95,1.0,14.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,49.0,156.5,20.0,82.2,87.3,147.7,81.7,82.0,78.0,43.0,60.0,119.0,4.39,120.0,0.0,360.0,2.0,NaN,NaN,NaN
102953,1.0,42.0,1.0,2.0,2.0,2.0,3.0,4.0,NaN,1.0,12.0,12.0,NaN,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,97.4,164.9,35.8,114.8,112.8,133.3,86.7,77.7,NaN,NaN,49.0,182.0,NaN,360.0,720.0,720.0,1.0,3.0,5.397605e-79,5.397605e-79
102954,2.0,41.0,4.0,2.0,1.0,1.0,5.0,5.0,2.0,7.0,10.0,10.0,1.18,1.0,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,69.1,162.6,26.1,86.4,102.7,121.3,73.0,NaN,49.0,108.0,54.0,172.0,9.49,600.0,0.0,60.0,2.0,NaN,NaN,NaN


In [492]:
# left merge the predictors into the target to ensure my mega_df is ready for a supervised learning task

mega_df = pd.merge(a1c, df_preds, how='left', on='SEQN')
mega_df

,glycohemoglobin,calculated_diagnosis,gender,age,race,veteran_status,country_of_birth,citizen_status,education,marital_status,pregnancy_status,household_size,annual_household_income,annual_family_income,income_poverty_ratio,coverage_status,covered_private,covered_medicare,covered_medigap,covered_medicaid,covered_chip,covered_military,covered_state,covered_other_gov,covered_single_service,not_covered,prescription_coverage,uninsured_in_last_year,weight_kg,height_cm,BMI,waist_circumference_cm,hip_circumference_cm,avg_systolic,avg_diastolic,avg_pulse,triglyceride,cholesterol_ldl,cholesterol_hdl,cholesterol_total,insulin,sedentary_minsperday,vigorous_activity_minsperweek,moderate_activity_minsperweek,lifetime_cigarette_smoker,current_cigarette_smoker,ecig_smoker,smokeless_tobacco_user
SEQN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
93705,6.2,Prediabetic,2.0,66.0,4.0,2.0,1.0,1.0,2.0,3.0,NaN,1.0,3.0,3.0,0.82,1.0,NaN,15.0,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,79.5,158.3,31.7,101.8,110.0,167.0,66.0,50.7,NaN,NaN,60.0,157.0,NaN,300.0,0.0,120.0,1.0,3.0,NaN,NaN
93706,5.2,Normal,1.0,18.0,6.0,2.0,1.0,1.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,66.3,175.7,21.5,79.3,94.4,129.0,71.0,77.3,NaN,NaN,47.0,148.0,NaN,240.0,0.0,285.0,2.0,NaN,NaN,NaN
93707,5.6,Normal,1.0,13.0,7.0,NaN,1.0,1.0,NaN,NaN,NaN,7.0,10.0,10.0,1.88,1.0,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,45.4,158.4,18.1,64.1,83.0,136.0,71.3,93.3,NaN,NaN,68.0,189.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93708,6.2,Prediabetic,2.0,66.0,6.0,2.0,2.0,1.0,1.0,1.0,NaN,2.0,6.0,6.0,1.63,1.0,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,53.5,150.2,23.7,88.2,90.1,146.3,79.7,67.7,58.0,109.0,88.0,209.0,9.72,120.0,0.0,150.0,2.0,NaN,NaN,NaN
93709,6.3,Prediabetic,2.0,75.0,4.0,2.0,1.0,1.0,4.0,2.0,NaN,1.0,2.0,2.0,0.41,1.0,NaN,15.0,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,88.8,151.1,38.9,113.0,131.3,119.0,79.3,62.3,NaN,NaN,65.0,176.0,NaN,600.0,0.0,360.0,1.0,1.0,5.397605e-79,5.397605e-79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102952,7.4,Diabetic,2.0,70.0,6.0,2.0,2.0,1.0,3.0,1.0,NaN,2.0,4.0,4.0,0.95,1.0,14.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,49.0,156.5,20.0,82.2,87.3,147.7,81.7,82.0,78.0,43.0,60.0,119.0,4.39,120.0,0.0,360.0,2.0,NaN,NaN,NaN
102953,5.9,Prediabetic,1.0,42.0,1.0,2.0,2.0,2.0,3.0,4.0,NaN,1.0,12.0,12.0,NaN,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,97.4,164.9,35.8,114.8,112.8,133.3,86.7,77.7,NaN,NaN,49.0,182.0,NaN,360.0,720.0,720.0,1.0,3.0,5.397605e-79,5.397605e-79
102954,5.2,Normal,2.0,41.0,4.0,2.0,1.0,1.0,5.0,5.0,2.0,7.0,10.0,10.0,1.18,1.0,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,69.1,162.6,26.1,86.4,102.7,121.3,73.0,NaN,49.0,108.0,54.0,172.0,9.49,600.0,0.0,60.0,2.0,NaN,NaN,NaN


In [493]:
mega_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6045 entries, 93705 to 102956
Data columns (total 48 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   glycohemoglobin                6045 non-null   float64
 1   calculated_diagnosis           6045 non-null   object 
 2   gender                         6045 non-null   float64
 3   age                            6045 non-null   float64
 4   race                           6045 non-null   float64
 5   veteran_status                 5399 non-null   float64
 6   country_of_birth               6045 non-null   float64
 7   citizen_status                 6042 non-null   float64
 8   education                      5018 non-null   float64
 9   marital_status                 5018 non-null   float64
 10  pregnancy_status               1000 non-null   float64
 11  household_size                 6045 non-null   float64
 12  annual_household_income        5763 non-nu

### Unencode Values, Deal with Nulls

In [494]:
def preprocessing_vcnulls(df, col):
    print(df[col].value_counts())
    print(f"Null count: {df[col].isna().sum()}")

In [495]:
preprocessing_vcnulls(mega_df, 'calculated_diagnosis')

Normal         3659
Prediabetic    1635
Diabetic        751
Name: calculated_diagnosis, dtype: int64
Null count: 0


In [496]:
preprocessing_vcnulls(mega_df, 'gender')

2.0    3111
1.0    2934
Name: gender, dtype: int64
Null count: 0


In [497]:
mega_df['gender'].replace({1.0: 'male', 2.0: 'female'}, inplace=True)

In [498]:
preprocessing_vcnulls(mega_df, 'age')

80.0    364
16.0    141
17.0    138
14.0    137
61.0    132
       ... 
72.0     52
76.0     45
77.0     45
79.0     43
78.0     41
Name: age, Length: 69, dtype: int64
Null count: 0


In [499]:
mega_df['age'] = mega_df['age'].astype(int)
mega_df['age'].value_counts()

80    364
16    141
17    138
14    137
61    132
     ... 
72     52
77     45
76     45
79     43
78     41
Name: age, Length: 69, dtype: int64

In [500]:
# focus only on adults
mega_df = mega_df[mega_df['age']>=18]

In [501]:
preprocessing_vcnulls(mega_df, 'age')

80    364
61    132
60    131
63    125
19    124
     ... 
72     52
76     45
77     45
79     43
78     41
Name: age, Length: 63, dtype: int64
Null count: 0


In [502]:
preprocessing_vcnulls(mega_df, 'race')

3.0    1829
4.0    1187
6.0     738
1.0     733
2.0     500
7.0     274
Name: race, dtype: int64
Null count: 0


In [503]:
mega_df['race'].replace({1.0: 'mexican_american',
                        2.0: 'hispanic',
                        3.0: 'white',
                        4.0: 'black',
                        6.0: 'asian',
                        7.0: 'other'}, inplace=True)

/Users/kristincooper/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [504]:
preprocessing_vcnulls(mega_df, 'race')

white               1829
black               1187
asian                738
mexican_american     733
hispanic             500
other                274
Name: race, dtype: int64
Null count: 0


In [505]:
preprocessing_vcnulls(mega_df, 'veteran_status')

2.0    4756
1.0     502
7.0       2
9.0       1
Name: veteran_status, dtype: int64
Null count: 0


In [506]:
# filled 3 inconclusive responses and 724 nulls with the mode
mega_df['veteran_status'].replace({1.0: 'yes',
                                  2.0: 'no',
                                  7.0: 'no',
                                  9.0: 'no',
                                  np.nan: 'no'}, inplace=True)

In [507]:
preprocessing_vcnulls(mega_df, 'veteran_status')

no     4759
yes     502
Name: veteran_status, dtype: int64
Null count: 0


In [508]:
preprocessing_vcnulls(mega_df, 'country_of_birth')

1.0     3648
2.0     1610
77.0       2
99.0       1
Name: country_of_birth, dtype: int64
Null count: 0


In [509]:
# filled 3 inconclusive responses with the mode
mega_df['country_of_birth'].replace({1.0: 'usa',
                                    2.0: 'other',
                                    77.0: 'usa',
                                    99.0: 'usa'}, inplace=True)

In [510]:
preprocessing_vcnulls(mega_df, 'country_of_birth')

usa      3651
other    1610
Name: country_of_birth, dtype: int64
Null count: 0


In [511]:
mega_df

,glycohemoglobin,calculated_diagnosis,gender,age,race,veteran_status,country_of_birth,citizen_status,education,marital_status,pregnancy_status,household_size,annual_household_income,annual_family_income,income_poverty_ratio,coverage_status,covered_private,covered_medicare,covered_medigap,covered_medicaid,covered_chip,covered_military,covered_state,covered_other_gov,covered_single_service,not_covered,prescription_coverage,uninsured_in_last_year,weight_kg,height_cm,BMI,waist_circumference_cm,hip_circumference_cm,avg_systolic,avg_diastolic,avg_pulse,triglyceride,cholesterol_ldl,cholesterol_hdl,cholesterol_total,insulin,sedentary_minsperday,vigorous_activity_minsperweek,moderate_activity_minsperweek,lifetime_cigarette_smoker,current_cigarette_smoker,ecig_smoker,smokeless_tobacco_user
SEQN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
93705,6.2,Prediabetic,female,66,black,no,usa,1.0,2.0,3.0,NaN,1.0,3.0,3.0,0.82,1.0,NaN,15.0,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,79.5,158.3,31.7,101.8,110.0,167.0,66.0,50.7,NaN,NaN,60.0,157.0,NaN,300.0,0.0,120.0,1.0,3.0,NaN,NaN
93706,5.2,Normal,male,18,asian,no,usa,1.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,66.3,175.7,21.5,79.3,94.4,129.0,71.0,77.3,NaN,NaN,47.0,148.0,NaN,240.0,0.0,285.0,2.0,NaN,NaN,NaN
93708,6.2,Prediabetic,female,66,asian,no,other,1.0,1.0,1.0,NaN,2.0,6.0,6.0,1.63,1.0,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,53.5,150.2,23.7,88.2,90.1,146.3,79.7,67.7,58.0,109.0,88.0,209.0,9.72,120.0,0.0,150.0,2.0,NaN,NaN,NaN
93709,6.3,Prediabetic,female,75,black,no,usa,1.0,4.0,2.0,NaN,1.0,2.0,2.0,0.41,1.0,NaN,15.0,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,88.8,151.1,38.9,113.0,131.3,119.0,79.3,62.3,NaN,NaN,65.0,176.0,NaN,600.0,0.0,360.0,1.0,1.0,5.397605e-79,5.397605e-79
93711,5.7,Prediabetic,male,56,asian,no,other,1.0,5.0,1.0,NaN,3.0,15.0,15.0,5.00,1.0,NaN,NaN,NaN,NaN,NaN,NaN,21.0,NaN,23.0,NaN,1.0,2.0,62.1,170.6,21.3,86.6,90.7,111.0,62.7,52.0,48.0,156.0,72.0,238.0,5.28,420.0,16.0,360.0,2.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102949,5.7,Prediabetic,male,33,white,no,usa,1.0,3.0,6.0,NaN,5.0,6.0,6.0,1.01,1.0,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,76.9,180.1,23.7,85.5,98.0,120.0,76.3,89.7,NaN,NaN,73.0,201.0,NaN,60.0,270.0,2730.0,1.0,1.0,NaN,5.397605e-79
102952,7.4,Diabetic,female,70,asian,no,other,1.0,3.0,1.0,NaN,2.0,4.0,4.0,0.95,1.0,14.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,49.0,156.5,20.0,82.2,87.3,147.7,81.7,82.0,78.0,43.0,60.0,119.0,4.39,120.0,0.0,360.0,2.0,NaN,NaN,NaN
102953,5.9,Prediabetic,male,42,mexican_american,no,other,2.0,3.0,4.0,NaN,1.0,12.0,12.0,NaN,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,97.4,164.9,35.8,114.8,112.8,133.3,86.7,77.7,NaN,NaN,49.0,182.0,NaN,360.0,720.0,720.0,1.0,3.0,5.397605e-79,5.397605e-79


In [512]:
preprocessing_vcnulls(mega_df, 'citizen_status')

1.0    4528
2.0     711
7.0      14
9.0       5
Name: citizen_status, dtype: int64
Null count: 3


In [513]:
# kept a "no_answer" class since this may be meaningful
mega_df['citizen_status'].replace({1.0: 'citizen',
                                  2.0: 'non_citizen',
                                  7.0: 'no_answer',
                                  9.0: 'no_answer',
                                  np.nan: 'no_answer'}, inplace=True)

/Users/kristincooper/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [514]:
preprocessing_vcnulls(mega_df, 'citizen_status')

citizen        4528
non_citizen     711
no_answer        22
Name: citizen_status, dtype: int64
Null count: 0


In [515]:
preprocessing_vcnulls(mega_df, 'education')

4.0    1622
3.0    1196
5.0    1187
2.0     572
1.0     430
9.0       9
7.0       2
Name: education, dtype: int64
Null count: 243


In [516]:
mega_df['education'].replace({1.0: 'no_diploma',
                             2.0: 'no_diploma',
                             3.0: 'highschool_grad',
                             4.0: 'some_college',
                             5.0: 'college_grad',
                             7.0: '0',
                             9.0: '0'}, inplace=True)

In [517]:
mega_df['education'].value_counts()

some_college       1622
highschool_grad    1196
college_grad       1187
no_diploma         1002
0                    11
Name: education, dtype: int64

In [518]:
mega_df[mega_df['education'].isna()].sort_values('age')

,glycohemoglobin,calculated_diagnosis,gender,age,race,veteran_status,country_of_birth,citizen_status,education,marital_status,pregnancy_status,household_size,annual_household_income,annual_family_income,income_poverty_ratio,coverage_status,covered_private,covered_medicare,covered_medigap,covered_medicaid,covered_chip,covered_military,covered_state,covered_other_gov,covered_single_service,not_covered,prescription_coverage,uninsured_in_last_year,weight_kg,height_cm,BMI,waist_circumference_cm,hip_circumference_cm,avg_systolic,avg_diastolic,avg_pulse,triglyceride,cholesterol_ldl,cholesterol_hdl,cholesterol_total,insulin,sedentary_minsperday,vigorous_activity_minsperweek,moderate_activity_minsperweek,lifetime_cigarette_smoker,current_cigarette_smoker,ecig_smoker,smokeless_tobacco_user
SEQN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
93706,5.2,Normal,male,18,asian,no,usa,citizen,NaN,NaN,NaN,5.0,NaN,NaN,NaN,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,66.3,175.7,21.5,79.3,94.4,129.0,71.0,77.3,NaN,NaN,47.0,148.0,NaN,240.0,0.0,285.0,2.0,NaN,NaN,NaN
98352,5.2,Normal,male,18,white,no,usa,citizen,NaN,NaN,NaN,4.0,14.0,14.0,3.19,1.0,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,62.2,178.8,19.5,71.2,92.0,116.3,66.0,61.0,NaN,NaN,61.0,129.0,NaN,360.0,45.0,375.0,2.0,NaN,1.500000e+01,5.397605e-79
98411,5.2,Normal,female,18,other,no,other,non_citizen,NaN,NaN,NaN,3.0,14.0,14.0,4.60,1.0,NaN,NaN,NaN,NaN,NaN,19.0,NaN,NaN,NaN,NaN,1.0,2.0,93.4,166.1,33.9,99.2,116.8,93.0,65.7,88.7,NaN,NaN,53.0,252.0,NaN,300.0,4.0,220.0,2.0,NaN,NaN,NaN
98427,5.4,Normal,male,18,asian,no,usa,citizen,NaN,NaN,NaN,4.0,10.0,10.0,2.59,1.0,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,55.4,165.8,20.2,70.8,85.3,100.0,66.3,90.7,NaN,NaN,72.0,171.0,NaN,480.0,0.0,250.0,2.0,NaN,5.397605e-79,NaN
98493,5.1,Normal,female,18,mexican_american,no,usa,citizen,NaN,NaN,NaN,4.0,5.0,5.0,0.80,1.0,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,65.7,157.0,26.7,89.2,100.2,112.3,72.3,92.7,NaN,NaN,58.0,149.0,NaN,600.0,16.0,1500.0,2.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95481,5.2,Normal,male,19,white,no,usa,citizen,NaN,NaN,NaN,4.0,9.0,9.0,2.29,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.8,172.4,22.5,78.4,93.7,117.3,63.3,73.0,NaN,NaN,48.0,167.0,NaN,240.0,189.0,1080.0,1.0,3.0,2.000000e+00,NaN
95463,5.3,Normal,male,19,mexican_american,no,other,non_citizen,NaN,NaN,NaN,7.0,99.0,99.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.7,167.9,35.0,112.9,108.1,122.3,79.3,63.3,236.0,118.0,32.0,197.0,24.43,240.0,0.0,70.0,2.0,NaN,NaN,NaN
98440,5.7,Prediabetic,female,19,black,no,usa,citizen,NaN,NaN,NaN,6.0,5.0,5.0,0.74,1.0,NaN,NaN,NaN,NaN,NaN,NaN,21.0,NaN,NaN,NaN,1.0,2.0,47.7,154.6,20.0,64.7,87.6,112.7,69.0,68.3,37.0,74.0,59.0,140.0,3.86,300.0,489.0,150.0,2.0,NaN,1.000000e+00,NaN


All null values for education are 18 and 19 year olds.

In [519]:
mega_df[mega_df['age']==18].sort_values('education')

,glycohemoglobin,calculated_diagnosis,gender,age,race,veteran_status,country_of_birth,citizen_status,education,marital_status,pregnancy_status,household_size,annual_household_income,annual_family_income,income_poverty_ratio,coverage_status,covered_private,covered_medicare,covered_medigap,covered_medicaid,covered_chip,covered_military,covered_state,covered_other_gov,covered_single_service,not_covered,prescription_coverage,uninsured_in_last_year,weight_kg,height_cm,BMI,waist_circumference_cm,hip_circumference_cm,avg_systolic,avg_diastolic,avg_pulse,triglyceride,cholesterol_ldl,cholesterol_hdl,cholesterol_total,insulin,sedentary_minsperday,vigorous_activity_minsperweek,moderate_activity_minsperweek,lifetime_cigarette_smoker,current_cigarette_smoker,ecig_smoker,smokeless_tobacco_user
SEQN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
93706,5.2,Normal,male,18,asian,no,usa,citizen,NaN,NaN,NaN,5.0,NaN,NaN,NaN,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,66.3,175.7,21.5,79.3,94.4,129.0,71.0,77.3,NaN,NaN,47.0,148.0,NaN,240.0,0.0,285.0,2.0,NaN,NaN,NaN
93712,5.4,Normal,male,18,mexican_american,no,other,non_citizen,NaN,NaN,NaN,4.0,4.0,4.0,0.76,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.9,172.8,19.7,72.0,88.2,115.0,58.0,74.0,NaN,NaN,48.0,182.0,NaN,120.0,2536.0,1350.0,1.0,2.0,5.397605e-79,NaN
93938,5.3,Normal,female,18,other,no,usa,citizen,NaN,NaN,NaN,3.0,10.0,10.0,3.37,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,63.1,163.2,23.7,92.1,94.6,104.0,59.3,76.7,NaN,NaN,63.0,177.0,NaN,120.0,25.0,1200.0,2.0,NaN,NaN,NaN
94001,5.1,Normal,female,18,mexican_american,no,usa,citizen,NaN,NaN,NaN,4.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,51.6,158.6,20.5,75.8,92.5,NaN,NaN,NaN,52.0,60.0,50.0,120.0,7.72,240.0,0.0,0.0,2.0,NaN,NaN,NaN
94144,5.6,Normal,male,18,other,no,usa,citizen,NaN,NaN,NaN,3.0,6.0,6.0,1.64,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122.6,173.5,40.7,127.4,125.2,104.7,67.7,65.0,NaN,NaN,36.0,138.0,NaN,360.0,0.0,1560.0,2.0,NaN,5.397605e-79,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102330,5.4,Normal,male,18,mexican_american,no,usa,citizen,NaN,NaN,NaN,5.0,7.0,7.0,1.50,1.0,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,65.6,172.4,22.1,74.6,91.8,115.7,71.0,68.7,NaN,NaN,61.0,141.0,NaN,420.0,2409.0,1440.0,2.0,NaN,1.000000e+00,5.397605e-79
102335,5.4,Normal,male,18,asian,no,usa,citizen,NaN,NaN,NaN,4.0,15.0,15.0,4.52,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,NaN,NaN,1.0,2.0,71.3,164.5,26.3,91.5,94.5,115.7,74.0,87.3,NaN,NaN,39.0,154.0,NaN,540.0,4.0,240.0,2.0,NaN,NaN,NaN
102366,5.2,Normal,female,18,mexican_american,no,other,non_citizen,NaN,NaN,NaN,4.0,8.0,8.0,2.03,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.5,159.8,31.1,106.3,110.5,101.7,61.7,NaN,NaN,NaN,33.0,160.0,NaN,480.0,9.0,135.0,2.0,NaN,NaN,NaN


All 18 year olds have null values for education.

In [520]:
mega_df[mega_df['age']==19].sort_values('education')

,glycohemoglobin,calculated_diagnosis,gender,age,race,veteran_status,country_of_birth,citizen_status,education,marital_status,pregnancy_status,household_size,annual_household_income,annual_family_income,income_poverty_ratio,coverage_status,covered_private,covered_medicare,covered_medigap,covered_medicaid,covered_chip,covered_military,covered_state,covered_other_gov,covered_single_service,not_covered,prescription_coverage,uninsured_in_last_year,weight_kg,height_cm,BMI,waist_circumference_cm,hip_circumference_cm,avg_systolic,avg_diastolic,avg_pulse,triglyceride,cholesterol_ldl,cholesterol_hdl,cholesterol_total,insulin,sedentary_minsperday,vigorous_activity_minsperweek,moderate_activity_minsperweek,lifetime_cigarette_smoker,current_cigarette_smoker,ecig_smoker,smokeless_tobacco_user
SEQN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
93765,5.2,Normal,male,19,white,no,usa,citizen,NaN,NaN,NaN,7.0,14.0,14.0,2.01,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,63.1,169.9,21.9,79.0,94.2,99.7,66.3,66.3,55.0,83.0,41.0,135.0,7.23,240.0,0.0,330.0,1.0,3.0,5.397605e-79,30.0
93778,5.5,Normal,female,19,black,no,usa,citizen,NaN,NaN,NaN,3.0,15.0,15.0,5.00,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,73.3,163.6,27.4,84.4,103.1,100.3,63.3,64.7,NaN,NaN,48.0,164.0,NaN,180.0,4.0,300.0,2.0,NaN,NaN,NaN
93858,5.2,Normal,female,19,other,no,usa,citizen,NaN,NaN,NaN,5.0,2.0,2.0,0.33,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.2,147.3,16.7,56.4,79.8,91.3,58.7,96.7,NaN,NaN,69.0,148.0,NaN,360.0,0.0,40.0,2.0,NaN,NaN,NaN
93884,4.8,Normal,male,19,white,no,usa,citizen,NaN,NaN,NaN,5.0,99.0,99.0,NaN,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,63.7,172.9,21.3,75.1,92.4,110.3,54.7,53.7,72.0,94.0,56.0,164.0,6.20,600.0,16.0,900.0,2.0,NaN,5.397605e-79,NaN
94078,4.6,Normal,female,19,white,no,usa,citizen,NaN,NaN,NaN,4.0,NaN,NaN,NaN,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,NaN,1.0,2.0,78.9,173.4,26.2,89.9,108.5,117.0,71.7,NaN,NaN,NaN,37.0,172.0,NaN,480.0,1.0,30.0,2.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102323,5.3,Normal,female,19,hispanic,no,other,citizen,NaN,NaN,NaN,5.0,7.0,7.0,1.19,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.2,155.5,19.9,73.9,88.3,99.7,69.7,83.3,46.0,52.0,55.0,116.0,6.09,180.0,0.0,0.0,2.0,NaN,2.000000e+00,NaN
102416,5.3,Normal,female,19,mexican_american,no,usa,citizen,NaN,NaN,NaN,6.0,10.0,10.0,2.00,1.0,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,82.0,170.0,28.4,102.0,109.8,109.0,63.7,73.3,43.0,100.0,63.0,172.0,12.94,600.0,0.0,0.0,2.0,NaN,NaN,NaN
102431,5.1,Normal,male,19,hispanic,no,usa,citizen,NaN,NaN,NaN,3.0,12.0,5.0,1.45,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.2,178.7,27.0,86.0,102.4,111.0,69.0,71.7,42.0,98.0,44.0,150.0,12.42,420.0,0.0,0.0,2.0,NaN,5.397605e-79,NaN


All 19 year olds have null values for education.


I will fill null education values with 0, then consider using the IterativeImputer() to fill them as a function of other features.

In [521]:
mega_df['education'].fillna('0', inplace=True)

/Users/kristincooper/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [522]:
preprocessing_vcnulls(mega_df, 'education')

some_college       1622
highschool_grad    1196
college_grad       1187
no_diploma         1002
0                   254
Name: education, dtype: int64
Null count: 0


In [523]:
preprocessing_vcnulls(mega_df, 'marital_status')

1.0     2508
5.0      895
3.0      571
6.0      450
2.0      407
4.0      182
77.0       5
Name: marital_status, dtype: int64
Null count: 243


In [524]:
mega_df['marital_status'].replace({1.0: 'married or living with partner',
                                  2.0: 'widowed',
                                  3.0: 'divorced/separated',
                                  4.0: 'divorced/separated',
                                  5.0: 'never married',
                                  6.0: 'married or living with partner',
                                  77.0: '0'}, inplace=True)

/Users/kristincooper/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [525]:
preprocessing_vcnulls(mega_df, 'marital_status')

married or living with partner    2958
never married                      895
divorced/separated                 753
widowed                            407
0                                    5
Name: marital_status, dtype: int64
Null count: 243


In [526]:
mega_df[mega_df['marital_status'].isna()].sort_values('age')

,glycohemoglobin,calculated_diagnosis,gender,age,race,veteran_status,country_of_birth,citizen_status,education,marital_status,pregnancy_status,household_size,annual_household_income,annual_family_income,income_poverty_ratio,coverage_status,covered_private,covered_medicare,covered_medigap,covered_medicaid,covered_chip,covered_military,covered_state,covered_other_gov,covered_single_service,not_covered,prescription_coverage,uninsured_in_last_year,weight_kg,height_cm,BMI,waist_circumference_cm,hip_circumference_cm,avg_systolic,avg_diastolic,avg_pulse,triglyceride,cholesterol_ldl,cholesterol_hdl,cholesterol_total,insulin,sedentary_minsperday,vigorous_activity_minsperweek,moderate_activity_minsperweek,lifetime_cigarette_smoker,current_cigarette_smoker,ecig_smoker,smokeless_tobacco_user
SEQN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
93706,5.2,Normal,male,18,asian,no,usa,citizen,0,NaN,NaN,5.0,NaN,NaN,NaN,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,66.3,175.7,21.5,79.3,94.4,129.0,71.0,77.3,NaN,NaN,47.0,148.0,NaN,240.0,0.0,285.0,2.0,NaN,NaN,NaN
98352,5.2,Normal,male,18,white,no,usa,citizen,0,NaN,NaN,4.0,14.0,14.0,3.19,1.0,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,62.2,178.8,19.5,71.2,92.0,116.3,66.0,61.0,NaN,NaN,61.0,129.0,NaN,360.0,45.0,375.0,2.0,NaN,1.500000e+01,5.397605e-79
98411,5.2,Normal,female,18,other,no,other,non_citizen,0,NaN,NaN,3.0,14.0,14.0,4.60,1.0,NaN,NaN,NaN,NaN,NaN,19.0,NaN,NaN,NaN,NaN,1.0,2.0,93.4,166.1,33.9,99.2,116.8,93.0,65.7,88.7,NaN,NaN,53.0,252.0,NaN,300.0,4.0,220.0,2.0,NaN,NaN,NaN
98427,5.4,Normal,male,18,asian,no,usa,citizen,0,NaN,NaN,4.0,10.0,10.0,2.59,1.0,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,55.4,165.8,20.2,70.8,85.3,100.0,66.3,90.7,NaN,NaN,72.0,171.0,NaN,480.0,0.0,250.0,2.0,NaN,5.397605e-79,NaN
98493,5.1,Normal,female,18,mexican_american,no,usa,citizen,0,NaN,NaN,4.0,5.0,5.0,0.80,1.0,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,65.7,157.0,26.7,89.2,100.2,112.3,72.3,92.7,NaN,NaN,58.0,149.0,NaN,600.0,16.0,1500.0,2.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95481,5.2,Normal,male,19,white,no,usa,citizen,0,NaN,NaN,4.0,9.0,9.0,2.29,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.8,172.4,22.5,78.4,93.7,117.3,63.3,73.0,NaN,NaN,48.0,167.0,NaN,240.0,189.0,1080.0,1.0,3.0,2.000000e+00,NaN
95463,5.3,Normal,male,19,mexican_american,no,other,non_citizen,0,NaN,NaN,7.0,99.0,99.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.7,167.9,35.0,112.9,108.1,122.3,79.3,63.3,236.0,118.0,32.0,197.0,24.43,240.0,0.0,70.0,2.0,NaN,NaN,NaN
98440,5.7,Prediabetic,female,19,black,no,usa,citizen,0,NaN,NaN,6.0,5.0,5.0,0.74,1.0,NaN,NaN,NaN,NaN,NaN,NaN,21.0,NaN,NaN,NaN,1.0,2.0,47.7,154.6,20.0,64.7,87.6,112.7,69.0,68.3,37.0,74.0,59.0,140.0,3.86,300.0,489.0,150.0,2.0,NaN,1.000000e+00,NaN


All null values in the marital status column are 18 or 19 year olds, so I will fill the nulls with "never married."

In [527]:
mega_df['marital_status'].fillna('never married', inplace=True)

/Users/kristincooper/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [528]:
preprocessing_vcnulls(mega_df, 'marital_status')

married or living with partner    2958
never married                     1138
divorced/separated                 753
widowed                            407
0                                    5
Name: marital_status, dtype: int64
Null count: 0


In [529]:
preprocessing_vcnulls(mega_df, 'pregnancy_status')

2.0    931
1.0     47
3.0     22
Name: pregnancy_status, dtype: int64
Null count: 4261


Per the data source, respondents were coded with a value 3 (could not be determined) in two scenarios:
- If the respondent reported did not know her pregnancy status and the urine test was negative
- If the respondent was interviewed but not examined 

Likely the second scenario has been removed from this subset. Because the urine test was negative, I am recoding all 3's as "not pregnant" and therefore leaving them in the dataset.

I also assume that null values indicate "not pregnant".

In [530]:
# drop individuals who are pregnant since their body measures may influence a model
mega_df = mega_df[mega_df['pregnancy_status'] != 1.0]

In [531]:
# remove 'pregnancy_status' column from dataset since now all values should be the same
mega_df.drop(columns='pregnancy_status', inplace=True)

/Users/kristincooper/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [532]:
mega_df

,glycohemoglobin,calculated_diagnosis,gender,age,race,veteran_status,country_of_birth,citizen_status,education,marital_status,household_size,annual_household_income,annual_family_income,income_poverty_ratio,coverage_status,covered_private,covered_medicare,covered_medigap,covered_medicaid,covered_chip,covered_military,covered_state,covered_other_gov,covered_single_service,not_covered,prescription_coverage,uninsured_in_last_year,weight_kg,height_cm,BMI,waist_circumference_cm,hip_circumference_cm,avg_systolic,avg_diastolic,avg_pulse,triglyceride,cholesterol_ldl,cholesterol_hdl,cholesterol_total,insulin,sedentary_minsperday,vigorous_activity_minsperweek,moderate_activity_minsperweek,lifetime_cigarette_smoker,current_cigarette_smoker,ecig_smoker,smokeless_tobacco_user
SEQN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
93705,6.2,Prediabetic,female,66,black,no,usa,citizen,no_diploma,divorced/separated,1.0,3.0,3.0,0.82,1.0,NaN,15.0,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,79.5,158.3,31.7,101.8,110.0,167.0,66.0,50.7,NaN,NaN,60.0,157.0,NaN,300.0,0.0,120.0,1.0,3.0,NaN,NaN
93706,5.2,Normal,male,18,asian,no,usa,citizen,0,never married,5.0,NaN,NaN,NaN,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,66.3,175.7,21.5,79.3,94.4,129.0,71.0,77.3,NaN,NaN,47.0,148.0,NaN,240.0,0.0,285.0,2.0,NaN,NaN,NaN
93708,6.2,Prediabetic,female,66,asian,no,other,citizen,no_diploma,married or living with partner,2.0,6.0,6.0,1.63,1.0,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,53.5,150.2,23.7,88.2,90.1,146.3,79.7,67.7,58.0,109.0,88.0,209.0,9.72,120.0,0.0,150.0,2.0,NaN,NaN,NaN
93709,6.3,Prediabetic,female,75,black,no,usa,citizen,some_college,widowed,1.0,2.0,2.0,0.41,1.0,NaN,15.0,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,88.8,151.1,38.9,113.0,131.3,119.0,79.3,62.3,NaN,NaN,65.0,176.0,NaN,600.0,0.0,360.0,1.0,1.0,5.397605e-79,5.397605e-79
93711,5.7,Prediabetic,male,56,asian,no,other,citizen,college_grad,married or living with partner,3.0,15.0,15.0,5.00,1.0,NaN,NaN,NaN,NaN,NaN,NaN,21.0,NaN,23.0,NaN,1.0,2.0,62.1,170.6,21.3,86.6,90.7,111.0,62.7,52.0,48.0,156.0,72.0,238.0,5.28,420.0,16.0,360.0,2.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102949,5.7,Prediabetic,male,33,white,no,usa,citizen,highschool_grad,married or living with partner,5.0,6.0,6.0,1.01,1.0,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,76.9,180.1,23.7,85.5,98.0,120.0,76.3,89.7,NaN,NaN,73.0,201.0,NaN,60.0,270.0,2730.0,1.0,1.0,NaN,5.397605e-79
102952,7.4,Diabetic,female,70,asian,no,other,citizen,highschool_grad,married or living with partner,2.0,4.0,4.0,0.95,1.0,14.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,49.0,156.5,20.0,82.2,87.3,147.7,81.7,82.0,78.0,43.0,60.0,119.0,4.39,120.0,0.0,360.0,2.0,NaN,NaN,NaN
102953,5.9,Prediabetic,male,42,mexican_american,no,other,non_citizen,highschool_grad,divorced/separated,1.0,12.0,12.0,NaN,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,97.4,164.9,35.8,114.8,112.8,133.3,86.7,77.7,NaN,NaN,49.0,182.0,NaN,360.0,720.0,720.0,1.0,3.0,5.397605e-79,5.397605e-79


Annual household income is the only income-related feature I will keep from the survey data, as it should most accurately represent the living situation of the individual. Please find below a description of how this was calculated.

For this model, I have recoded values as the upper end of each income range reported in order to create a continuous numeric variable. The 100k+ category was coded as 150,000 in keeping with the pattern of higher annual incomes. If respondents only reported if their income was under 20k (197 total respondents), I filled their income as 15,000 since this is likely a larger range with more actual values falling below the maximum. Similarly, if the respondent only reported if their income was over 20k (69 total respondents, I filled their income as 45000.

From data source:

> If a "household" was comprised of a single family or individual, the reported family income was used as household income as well. When more than one family, or one or more unrelated individuals, or a combination of a family and unrelated individuals resided in the household, the total household income was calculated by the sum of all reported family and/or individual income values. When more than one family, or one or more unrelated individuals, or a combination of a family and unrelated individuals resided in the same household, they were asked to provide a total income estimate for the entire household, using similar questions as were used for family income. This estimated household income value was only used when: 1) the family income value was missing for one or more families in the household; and 2) the estimated value was equal or more than the sum of all known family incomes from the household. If different respondents in the household provided different estimates, the largest value was used. If none of the respondents provided a valid household income estimate, but the sum of known family and/or individual incomes was at least 100,000, then household income was categorized as “100,000 and over.”

> If the respondent was not willing or able to provide an exact dollar figure, the interviewer asked an additional question to determine whether the income was < 20,000 or ≥ 20,000. Based on the respondent’s answer to this question, he/she was asked to select a category of income from a list on a hand card. For respondents who selected a category of income, their family incomes were set as the midpoints of the selected ranges. If the respondent was unable to report greater detail than < 20,000 or ≥ 20,000, then these two categories were used to report the family (or individual) income.



In [533]:
preprocessing_vcnulls(mega_df, 'annual_household_income')

15.0    903
6.0     523
7.0     517
14.0    464
8.0     345
5.0     321
4.0     306
9.0     295
10.0    250
3.0     245
12.0    197
2.0     150
1.0     144
99.0    125
77.0    101
13.0     69
Name: annual_household_income, dtype: int64
Null count: 259


In [534]:
mega_df['annual_household_income'].replace({1.0: 5000,
                                           2.0: 10000,
                                           3.0: 15000,
                                           4.0: 20000,
                                           5.0: 25000,
                                           6.0: 35000,
                                           7.0: 45000,
                                           8.0: 55000,
                                           9.0: 65000,
                                           10.0: 75000,
                                           12.0: 15000,
                                           13.0: 45000,
                                           14.0: 100000,
                                           15.0: 150000,
                                           77.0: 0,
                                           99.0: 0}, inplace=True)

/Users/kristincooper/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [535]:
mega_df[mega_df['annual_household_income'].isna()].sort_values('age')

,glycohemoglobin,calculated_diagnosis,gender,age,race,veteran_status,country_of_birth,citizen_status,education,marital_status,household_size,annual_household_income,annual_family_income,income_poverty_ratio,coverage_status,covered_private,covered_medicare,covered_medigap,covered_medicaid,covered_chip,covered_military,covered_state,covered_other_gov,covered_single_service,not_covered,prescription_coverage,uninsured_in_last_year,weight_kg,height_cm,BMI,waist_circumference_cm,hip_circumference_cm,avg_systolic,avg_diastolic,avg_pulse,triglyceride,cholesterol_ldl,cholesterol_hdl,cholesterol_total,insulin,sedentary_minsperday,vigorous_activity_minsperweek,moderate_activity_minsperweek,lifetime_cigarette_smoker,current_cigarette_smoker,ecig_smoker,smokeless_tobacco_user
SEQN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
93706,5.2,Normal,male,18,asian,no,usa,citizen,0,never married,5.0,NaN,NaN,NaN,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,66.3,175.7,21.5,79.3,94.4,129.0,71.0,77.3,NaN,NaN,47.0,148.0,NaN,240.0,0.0,285.0,2.0,NaN,NaN,NaN
94545,5.5,Normal,male,18,asian,no,usa,citizen,0,never married,3.0,NaN,NaN,NaN,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,90.2,180.0,27.8,97.8,107.8,95.0,62.7,61.3,NaN,NaN,33.0,129.0,NaN,420.0,36.0,240.0,2.0,NaN,3.000000e+00,NaN
94001,5.1,Normal,female,18,mexican_american,no,usa,citizen,0,never married,4.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,51.6,158.6,20.5,75.8,92.5,NaN,NaN,NaN,52.0,60.0,50.0,120.0,7.72,240.0,0.0,0.0,2.0,NaN,NaN,NaN
99015,5.1,Normal,male,18,mexican_american,no,other,non_citizen,0,never married,2.0,NaN,1.0,0.37,1.0,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,74.0,170.1,25.6,89.4,99.0,118.3,65.3,67.0,NaN,NaN,43.0,114.0,NaN,240.0,265.0,240.0,1.0,3.0,5.397605e-79,1.0
95269,4.6,Normal,male,18,black,no,other,non_citizen,0,never married,4.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.4,177.0,17.7,68.3,86.4,NaN,NaN,NaN,NaN,NaN,72.0,166.0,NaN,120.0,0.0,0.0,2.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100480,5.6,Normal,male,80,hispanic,no,other,non_citizen,no_diploma,married or living with partner,5.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.1,156.8,25.3,94.8,93.1,146.3,76.3,65.0,NaN,NaN,41.0,195.0,NaN,240.0,0.0,0.0,1.0,3.0,NaN,NaN
100914,5.6,Normal,female,80,white,no,usa,citizen,highschool_grad,widowed,3.0,NaN,NaN,NaN,1.0,14.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,86.3,161.0,33.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.0,170.0,NaN,660.0,0.0,0.0,2.0,NaN,NaN,NaN
94575,5.3,Normal,female,80,white,no,usa,citizen,highschool_grad,never married,1.0,NaN,NaN,NaN,1.0,14.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,54.9,161.4,21.1,80.0,97.0,151.3,83.3,75.3,NaN,NaN,90.0,255.0,NaN,240.0,0.0,0.0,2.0,NaN,NaN,NaN


Couldn't find a pattern in the missing household income values, so I'm coding them as 0 for now and will consider using either SimpleImputer or IterativeImputer to fill them after I've cleaned up other missing values.

In [536]:
mega_df['annual_household_income'].fillna(0, inplace=True)

/Users/kristincooper/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [537]:
mega_df['annual_household_income'] = mega_df['annual_household_income'].astype(int)

<ipython-input-537-7a7865e89efb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mega_df['annual_household_income'] = mega_df['annual_household_income'].astype(int)


In [538]:
preprocessing_vcnulls(mega_df, 'annual_household_income')

150000    903
45000     586
35000     523
0         485
100000    464
15000     442
55000     345
25000     321
20000     306
65000     295
75000     250
10000     150
5000      144
Name: annual_household_income, dtype: int64
Null count: 0


In [539]:
mega_df.drop(columns='annual_family_income', inplace=True)

/Users/kristincooper/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [540]:
preprocessing_vcnulls(mega_df, 'household_size')

2.0    1611
3.0     947
4.0     823
1.0     721
5.0     571
6.0     311
7.0     230
Name: household_size, dtype: int64
Null count: 0


In [541]:
mega_df['household_size'] = mega_df['household_size'].astype(int)

<ipython-input-541-b6a0761daefb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mega_df['household_size'] = mega_df['household_size'].astype(int)


**ratio_income_householdsize** feature was created by dividing the annual household income by the household size, which represents the total number of people in the household.

In [542]:
mega_df['ratio_income_householdsize'] = mega_df['annual_household_income'] / mega_df['household_size']
mega_df['ratio_income_householdsize'] = mega_df['ratio_income_householdsize'].astype(int)
mega_df

<ipython-input-542-6aa72fb31d11>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mega_df['ratio_income_householdsize'] = mega_df['annual_household_income'] / mega_df['household_size']
<ipython-input-542-6aa72fb31d11>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mega_df['ratio_income_householdsize'] = mega_df['ratio_income_householdsize'].astype(int)


,glycohemoglobin,calculated_diagnosis,gender,age,race,veteran_status,country_of_birth,citizen_status,education,marital_status,household_size,annual_household_income,income_poverty_ratio,coverage_status,covered_private,covered_medicare,covered_medigap,covered_medicaid,covered_chip,covered_military,covered_state,covered_other_gov,covered_single_service,not_covered,prescription_coverage,uninsured_in_last_year,weight_kg,height_cm,BMI,waist_circumference_cm,hip_circumference_cm,avg_systolic,avg_diastolic,avg_pulse,triglyceride,cholesterol_ldl,cholesterol_hdl,cholesterol_total,insulin,sedentary_minsperday,vigorous_activity_minsperweek,moderate_activity_minsperweek,lifetime_cigarette_smoker,current_cigarette_smoker,ecig_smoker,smokeless_tobacco_user,ratio_income_householdsize
SEQN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
93705,6.2,Prediabetic,female,66,black,no,usa,citizen,no_diploma,divorced/separated,1,15000,0.82,1.0,NaN,15.0,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,79.5,158.3,31.7,101.8,110.0,167.0,66.0,50.7,NaN,NaN,60.0,157.0,NaN,300.0,0.0,120.0,1.0,3.0,NaN,NaN,15000
93706,5.2,Normal,male,18,asian,no,usa,citizen,0,never married,5,0,NaN,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,66.3,175.7,21.5,79.3,94.4,129.0,71.0,77.3,NaN,NaN,47.0,148.0,NaN,240.0,0.0,285.0,2.0,NaN,NaN,NaN,0
93708,6.2,Prediabetic,female,66,asian,no,other,citizen,no_diploma,married or living with partner,2,35000,1.63,1.0,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,53.5,150.2,23.7,88.2,90.1,146.3,79.7,67.7,58.0,109.0,88.0,209.0,9.72,120.0,0.0,150.0,2.0,NaN,NaN,NaN,17500
93709,6.3,Prediabetic,female,75,black,no,usa,citizen,some_college,widowed,1,10000,0.41,1.0,NaN,15.0,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,88.8,151.1,38.9,113.0,131.3,119.0,79.3,62.3,NaN,NaN,65.0,176.0,NaN,600.0,0.0,360.0,1.0,1.0,5.397605e-79,5.397605e-79,10000
93711,5.7,Prediabetic,male,56,asian,no,other,citizen,college_grad,married or living with partner,3,150000,5.00,1.0,NaN,NaN,NaN,NaN,NaN,NaN,21.0,NaN,23.0,NaN,1.0,2.0,62.1,170.6,21.3,86.6,90.7,111.0,62.7,52.0,48.0,156.0,72.0,238.0,5.28,420.0,16.0,360.0,2.0,NaN,NaN,NaN,50000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102949,5.7,Prediabetic,male,33,white,no,usa,citizen,highschool_grad,married or living with partner,5,35000,1.01,1.0,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,76.9,180.1,23.7,85.5,98.0,120.0,76.3,89.7,NaN,NaN,73.0,201.0,NaN,60.0,270.0,2730.0,1.0,1.0,NaN,5.397605e-79,7000
102952,7.4,Diabetic,female,70,asian,no,other,citizen,highschool_grad,married or living with partner,2,20000,0.95,1.0,14.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,49.0,156.5,20.0,82.2,87.3,147.7,81.7,82.0,78.0,43.0,60.0,119.0,4.39,120.0,0.0,360.0,2.0,NaN,NaN,NaN,10000
102953,5.9,Prediabetic,male,42,mexican_american,no,other,non_citizen,highschool_grad,divorced/separated,1,15000,NaN,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,97.4,164.9,35.8,114.8,112.8,133.3,86.7,77.7,NaN,NaN,49.0,182.0,NaN,360.0,720.0,720.0,1.0,3.0,5.397605e-79,5.397605e-79,15000


**Income Poverty Ratio** was calculated by dividing family (or individual) income by the poverty guidelines specific to the survey year. The value was not computed if the respondent only reported income as < 20,000 or ≥ 20,000. If family income was reported as a more detailed category, the midpoint of the range was used to compute the ratio. Values at or above 5.00 were coded as 5.00 or more because of disclosure concerns. The values were not computed if the income data was missing.

In [543]:
mega_df['income_poverty_ratio'].describe().round(3)

count    4536.000
mean        2.526
std         1.609
min         0.000
25%         1.180
50%         2.080
75%         4.070
max         5.000
Name: income_poverty_ratio, dtype: float64

In [544]:
mega_df['income_poverty_ratio'].isna().sum()

678

In [545]:
preprocessing_vcnulls(mega_df, 'coverage_status')

1.0    4409
2.0     791
9.0       9
7.0       5
Name: coverage_status, dtype: int64
Null count: 0


In [546]:
# kept a "no_answer" class as this may be meaningful
mega_df['coverage_status'].replace({1.0: 'insured',
                                   2.0: 'uninsured',
                                   7.0: 'no_answer',
                                   9.0: 'no_answer'}, inplace=True)

/Users/kristincooper/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [547]:
preprocessing_vcnulls(mega_df, 'coverage_status')

insured      4409
uninsured     791
no_answer      14
Name: coverage_status, dtype: int64
Null count: 0


In [548]:
preprocessing_vcnulls(mega_df, 'covered_private')

14.0    2548
99.0      37
77.0       2
Name: covered_private, dtype: int64
Null count: 2627


In [549]:
mega_df['covered_private'].replace({14.0: 'yes',
                                   99.0: 'no',
                                   77.0: 'no',
                                   np.nan: 'no'}, inplace=True)

In [550]:
preprocessing_vcnulls(mega_df, 'covered_medicare')

15.0    1402
Name: covered_medicare, dtype: int64
Null count: 3812


In [551]:
mega_df['covered_medicare'].replace({15.0: 'yes', np.nan: 'no'}, inplace=True)

In [552]:
preprocessing_vcnulls(mega_df, 'covered_medigap')

16.0    53
Name: covered_medigap, dtype: int64
Null count: 5161


In [553]:
mega_df['covered_medigap'].replace({16.0: 'yes', np.nan: 'no'}, inplace=True)

In [554]:
preprocessing_vcnulls(mega_df, 'covered_medicaid')

17.0    893
Name: covered_medicaid, dtype: int64
Null count: 4321


In [555]:
mega_df['covered_medicaid'].replace({17.0: 'yes', np.nan: 'no'}, inplace=True)

In [556]:
preprocessing_vcnulls(mega_df, 'covered_chip')

18.0    2
Name: covered_chip, dtype: int64
Null count: 5212


In [557]:
mega_df['covered_chip'].replace({18.0: 'yes', np.nan: 'no'}, inplace=True)

In [558]:
preprocessing_vcnulls(mega_df, 'covered_military')

19.0    229
Name: covered_military, dtype: int64
Null count: 4985


In [559]:
mega_df['covered_military'].replace({19.0: 'yes', np.nan: 'no'}, inplace=True)

In [560]:
preprocessing_vcnulls(mega_df, 'covered_state')

21.0    210
Name: covered_state, dtype: int64
Null count: 5004


In [561]:
mega_df['covered_state'].replace({21.0: 'yes', np.nan: 'no'}, inplace=True)

In [562]:
preprocessing_vcnulls(mega_df, 'covered_other_gov')

22.0    204
Name: covered_other_gov, dtype: int64
Null count: 5010


In [563]:
mega_df['covered_other_gov'].replace({22.0: 'yes', np.nan: 'no'}, inplace=True)

In [564]:
preprocessing_vcnulls(mega_df, 'covered_single_service')

23.0    472
Name: covered_single_service, dtype: int64
Null count: 4742


In [565]:
mega_df['covered_single_service'].replace({23.0: 'yes', np.nan: 'no'}, inplace=True)

In [566]:
preprocessing_vcnulls(mega_df, 'not_covered')

40.0    1
Name: not_covered, dtype: int64
Null count: 5213


In [567]:
#seems to be an anomoly, so I drop this row
mega_df[mega_df['not_covered']==40.0]
mega_df = mega_df[mega_df['not_covered']!=40.0]

In [568]:
mega_df.drop(columns='not_covered', inplace=True)

/Users/kristincooper/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [569]:
mega_df

,glycohemoglobin,calculated_diagnosis,gender,age,race,veteran_status,country_of_birth,citizen_status,education,marital_status,household_size,annual_household_income,income_poverty_ratio,coverage_status,covered_private,covered_medicare,covered_medigap,covered_medicaid,covered_chip,covered_military,covered_state,covered_other_gov,covered_single_service,prescription_coverage,uninsured_in_last_year,weight_kg,height_cm,BMI,waist_circumference_cm,hip_circumference_cm,avg_systolic,avg_diastolic,avg_pulse,triglyceride,cholesterol_ldl,cholesterol_hdl,cholesterol_total,insulin,sedentary_minsperday,vigorous_activity_minsperweek,moderate_activity_minsperweek,lifetime_cigarette_smoker,current_cigarette_smoker,ecig_smoker,smokeless_tobacco_user,ratio_income_householdsize
SEQN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
93705,6.2,Prediabetic,female,66,black,no,usa,citizen,no_diploma,divorced/separated,1,15000,0.82,insured,no,yes,no,yes,no,no,no,no,no,1.0,2.0,79.5,158.3,31.7,101.8,110.0,167.0,66.0,50.7,NaN,NaN,60.0,157.0,NaN,300.0,0.0,120.0,1.0,3.0,NaN,NaN,15000
93706,5.2,Normal,male,18,asian,no,usa,citizen,0,never married,5,0,NaN,insured,yes,no,no,no,no,no,no,no,no,1.0,2.0,66.3,175.7,21.5,79.3,94.4,129.0,71.0,77.3,NaN,NaN,47.0,148.0,NaN,240.0,0.0,285.0,2.0,NaN,NaN,NaN,0
93708,6.2,Prediabetic,female,66,asian,no,other,citizen,no_diploma,married or living with partner,2,35000,1.63,insured,no,yes,no,no,no,no,no,no,no,1.0,2.0,53.5,150.2,23.7,88.2,90.1,146.3,79.7,67.7,58.0,109.0,88.0,209.0,9.72,120.0,0.0,150.0,2.0,NaN,NaN,NaN,17500
93709,6.3,Prediabetic,female,75,black,no,usa,citizen,some_college,widowed,1,10000,0.41,insured,no,yes,no,yes,no,no,no,no,no,1.0,2.0,88.8,151.1,38.9,113.0,131.3,119.0,79.3,62.3,NaN,NaN,65.0,176.0,NaN,600.0,0.0,360.0,1.0,1.0,5.397605e-79,5.397605e-79,10000
93711,5.7,Prediabetic,male,56,asian,no,other,citizen,college_grad,married or living with partner,3,150000,5.00,insured,no,no,no,no,no,no,yes,no,yes,1.0,2.0,62.1,170.6,21.3,86.6,90.7,111.0,62.7,52.0,48.0,156.0,72.0,238.0,5.28,420.0,16.0,360.0,2.0,NaN,NaN,NaN,50000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102949,5.7,Prediabetic,male,33,white,no,usa,citizen,highschool_grad,married or living with partner,5,35000,1.01,insured,no,no,no,yes,no,no,no,no,no,1.0,2.0,76.9,180.1,23.7,85.5,98.0,120.0,76.3,89.7,NaN,NaN,73.0,201.0,NaN,60.0,270.0,2730.0,1.0,1.0,NaN,5.397605e-79,7000
102952,7.4,Diabetic,female,70,asian,no,other,citizen,highschool_grad,married or living with partner,2,20000,0.95,insured,yes,yes,no,no,no,no,no,no,no,1.0,2.0,49.0,156.5,20.0,82.2,87.3,147.7,81.7,82.0,78.0,43.0,60.0,119.0,4.39,120.0,0.0,360.0,2.0,NaN,NaN,NaN,10000
102953,5.9,Prediabetic,male,42,mexican_american,no,other,non_citizen,highschool_grad,divorced/separated,1,15000,NaN,insured,yes,no,no,no,no,no,no,no,no,1.0,1.0,97.4,164.9,35.8,114.8,112.8,133.3,86.7,77.7,NaN,NaN,49.0,182.0,NaN,360.0,720.0,720.0,1.0,3.0,5.397605e-79,5.397605e-79,15000


In [570]:
preprocessing_vcnulls(mega_df, 'prescription_coverage')

1.0    4068
2.0     274
9.0      76
7.0       1
Name: prescription_coverage, dtype: int64
Null count: 794


In [571]:
mega_df['prescription_coverage'].replace({1.0: 'yes',
                                         2.0: 'no',
                                         7.0: 'no_answer',
                                         9.0: 'no_answer',
                                         np.nan: 'no_answer'}, inplace=True)

/Users/kristincooper/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [572]:
preprocessing_vcnulls(mega_df, 'prescription_coverage')

yes          4068
no_answer     871
no            274
Name: prescription_coverage, dtype: int64
Null count: 0


In [573]:
preprocessing_vcnulls(mega_df, 'uninsured_in_last_year')

2.0    4092
1.0     320
9.0       7
Name: uninsured_in_last_year, dtype: int64
Null count: 794


In [574]:
mega_df['uninsured_in_last_year'].replace({1.0: 'yes',
                                          2.0: 'no',
                                          7.0: 'no_answer',
                                          9.0: 'no_answer'}, inplace=True)

In [575]:
mega_df[mega_df['uninsured_in_last_year'].isna()]

,glycohemoglobin,calculated_diagnosis,gender,age,race,veteran_status,country_of_birth,citizen_status,education,marital_status,household_size,annual_household_income,income_poverty_ratio,coverage_status,covered_private,covered_medicare,covered_medigap,covered_medicaid,covered_chip,covered_military,covered_state,covered_other_gov,covered_single_service,prescription_coverage,uninsured_in_last_year,weight_kg,height_cm,BMI,waist_circumference_cm,hip_circumference_cm,avg_systolic,avg_diastolic,avg_pulse,triglyceride,cholesterol_ldl,cholesterol_hdl,cholesterol_total,insulin,sedentary_minsperday,vigorous_activity_minsperweek,moderate_activity_minsperweek,lifetime_cigarette_smoker,current_cigarette_smoker,ecig_smoker,smokeless_tobacco_user,ratio_income_householdsize
SEQN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
93712,5.4,Normal,male,18,mexican_american,no,other,non_citizen,0,never married,4,20000,0.76,uninsured,no,no,no,no,no,no,no,no,no,no_answer,NaN,58.9,172.8,19.7,72.0,88.2,115.0,58.0,74.0,NaN,NaN,48.0,182.0,NaN,120.0,2536.0,1350.0,1.0,2.0,5.397605e-79,NaN,5000
93717,5.1,Normal,male,22,white,yes,usa,citizen,highschool_grad,never married,2,0,1.49,uninsured,no,no,no,no,no,no,no,no,no,no_answer,NaN,74.4,174.1,24.5,86.2,91.8,109.7,60.3,61.7,102.0,140.0,53.0,213.0,3.94,300.0,0.0,210.0,1.0,1.0,5.397605e-79,NaN,0
93729,5.3,Normal,male,42,black,no,usa,citizen,no_diploma,never married,2,35000,0.83,no_answer,no,no,no,no,no,no,no,no,no,no_answer,NaN,82.8,173.2,27.6,101.5,103.4,116.0,68.0,69.3,NaN,NaN,49.0,165.0,NaN,480.0,0.0,0.0,1.0,2.0,NaN,NaN,17500
93730,13.3,Diabetic,male,57,hispanic,no,other,citizen,no_diploma,widowed,1,0,NaN,uninsured,no,no,no,no,no,no,no,no,no,no_answer,NaN,80.2,167.6,28.6,96.3,97.5,NaN,NaN,NaN,NaN,NaN,49.0,221.0,NaN,180.0,1.0,160.0,2.0,NaN,NaN,NaN,0
93743,5.9,Prediabetic,male,61,white,no,usa,citizen,no_diploma,married or living with partner,2,15000,3.87,uninsured,no,no,no,no,no,no,no,no,no,no_answer,NaN,79.3,187.8,22.5,93.7,102.3,172.0,98.3,67.3,NaN,NaN,41.0,146.0,NaN,420.0,2520.0,840.0,1.0,1.0,1.000000e+00,3.000000e+01,7500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102832,5.1,Normal,male,29,white,no,usa,citizen,highschool_grad,married or living with partner,5,20000,0.65,uninsured,no,no,no,no,no,no,no,no,no,no_answer,NaN,65.5,173.9,21.7,86.2,94.2,124.7,81.3,70.3,38.0,89.0,52.0,149.0,3.23,120.0,150.0,600.0,1.0,3.0,5.397605e-79,5.397605e-79,4000
102879,4.4,Normal,male,21,white,no,usa,citizen,highschool_grad,never married,1,5000,0.04,uninsured,no,no,no,no,no,no,no,no,no,no_answer,NaN,122.3,169.6,42.5,130.5,125.9,106.7,70.7,78.3,NaN,NaN,38.0,138.0,NaN,300.0,0.0,50.0,2.0,NaN,NaN,NaN,5000
102884,5.4,Normal,male,32,mexican_american,no,other,non_citizen,highschool_grad,never married,1,35000,2.49,uninsured,no,no,no,no,no,no,no,no,no,no_answer,NaN,90.4,169.0,31.7,101.4,106.6,128.0,77.0,73.3,NaN,NaN,59.0,254.0,NaN,180.0,601.0,720.0,1.0,2.0,NaN,NaN,35000


In [576]:
for ind, row in mega_df.iterrows():
    if mega_df['coverage_status'][ind] == 'uninsured':
        mega_df['uninsured_in_last_year'][ind] = 'yes'

<ipython-input-576-ffe3f162f178>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mega_df['uninsured_in_last_year'][ind] = 'yes'
/Users/kristincooper/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3417: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [577]:
mega_df[mega_df['uninsured_in_last_year'].isna()]

,glycohemoglobin,calculated_diagnosis,gender,age,race,veteran_status,country_of_birth,citizen_status,education,marital_status,household_size,annual_household_income,income_poverty_ratio,coverage_status,covered_private,covered_medicare,covered_medigap,covered_medicaid,covered_chip,covered_military,covered_state,covered_other_gov,covered_single_service,prescription_coverage,uninsured_in_last_year,weight_kg,height_cm,BMI,waist_circumference_cm,hip_circumference_cm,avg_systolic,avg_diastolic,avg_pulse,triglyceride,cholesterol_ldl,cholesterol_hdl,cholesterol_total,insulin,sedentary_minsperday,vigorous_activity_minsperweek,moderate_activity_minsperweek,lifetime_cigarette_smoker,current_cigarette_smoker,ecig_smoker,smokeless_tobacco_user,ratio_income_householdsize
SEQN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
93729,5.3,Normal,male,42,black,no,usa,citizen,no_diploma,never married,2,35000,0.83,no_answer,no,no,no,no,no,no,no,no,no,no_answer,NaN,82.8,173.2,27.6,101.5,103.4,116.0,68.0,69.3,NaN,NaN,49.0,165.0,NaN,480.0,0.0,0.0,1.0,2.0,NaN,NaN,17500
93902,5.4,Normal,female,24,mexican_american,no,other,non_citizen,no_diploma,married or living with partner,7,0,NaN,no_answer,no,no,no,no,no,no,no,no,no,no_answer,NaN,68.8,158.2,27.5,92.4,100.3,NaN,NaN,NaN,NaN,NaN,35.0,165.0,NaN,120.0,0.0,150.0,2.0,NaN,NaN,NaN,0
93981,7.2,Diabetic,female,52,asian,no,other,non_citizen,college_grad,married or living with partner,3,0,NaN,no_answer,no,no,no,no,no,no,no,no,no,no_answer,NaN,66.3,157.3,26.8,93.4,100.0,118.0,63.3,55.3,129.0,171.0,45.0,242.0,8.72,240.0,0.0,90.0,2.0,NaN,NaN,NaN,0
95515,6.3,Prediabetic,male,69,black,no,usa,citizen,some_college,widowed,2,0,NaN,no_answer,no,no,no,no,no,no,no,no,no,no_answer,NaN,121.1,190.0,33.5,126.4,124.7,121.0,75.3,60.3,234.0,155.0,64.0,266.0,17.80,9999.0,4.0,0.0,1.0,3.0,NaN,NaN,0
95659,5.7,Prediabetic,male,46,black,no,usa,citizen,highschool_grad,never married,1,5000,0.04,no_answer,no,no,no,no,no,no,no,no,no,no_answer,NaN,86.2,180.0,26.6,102.3,106.9,121.0,82.0,77.3,NaN,NaN,NaN,NaN,NaN,240.0,0.0,0.0,1.0,3.0,NaN,NaN,5000
96934,4.8,Normal,female,55,black,no,usa,citizen,some_college,widowed,1,5000,0.04,no_answer,no,no,no,no,no,no,no,no,no,no_answer,NaN,39.9,160.6,15.5,69.1,77.7,106.7,69.7,87.7,98.0,99.0,77.0,196.0,6.30,600.0,0.0,1680.0,1.0,1.0,NaN,NaN,5000
97306,5.4,Normal,female,20,mexican_american,no,usa,no_answer,highschool_grad,never married,6,0,NaN,no_answer,no,no,no,no,no,no,no,no,no,no_answer,NaN,85.3,161.8,32.6,87.9,108.8,97.0,65.3,71.7,66.0,117.0,44.0,174.0,29.19,300.0,0.0,1260.0,2.0,NaN,NaN,NaN,0
97486,6.8,Diabetic,female,69,black,no,usa,citizen,0,0,1,0,NaN,no_answer,no,no,no,no,no,no,no,no,no,no_answer,NaN,94.1,156.4,38.5,120.5,128.7,NaN,NaN,NaN,NaN,NaN,57.0,174.0,NaN,120.0,0.0,0.0,2.0,NaN,NaN,NaN,0
97774,5.0,Normal,male,33,mexican_american,no,other,non_citizen,no_diploma,divorced/separated,2,0,NaN,no_answer,no,no,no,no,no,no,no,no,no,no_answer,NaN,51.0,158.5,20.3,73.2,83.4,149.3,82.3,72.3,125.0,94.0,60.0,179.0,6.36,120.0,3289.0,480.0,2.0,NaN,NaN,NaN,0


In [578]:
mega_df['uninsured_in_last_year'].fillna('no_answer', inplace=True)

/Users/kristincooper/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [579]:
preprocessing_vcnulls(mega_df, 'uninsured_in_last_year')

no           4082
yes          1110
no_answer      21
Name: uninsured_in_last_year, dtype: int64
Null count: 0


In [580]:
mega_df['weight_kg'].describe().round(1)

count    5136.0
mean       82.5
std        22.9
min        32.6
25%        66.5
50%        79.0
75%        94.6
max       242.6
Name: weight_kg, dtype: float64

In [581]:
mega_df['weight_kg'].isna().sum()

77

In [582]:
mega_df[mega_df['weight_kg'].isna()]

,glycohemoglobin,calculated_diagnosis,gender,age,race,veteran_status,country_of_birth,citizen_status,education,marital_status,household_size,annual_household_income,income_poverty_ratio,coverage_status,covered_private,covered_medicare,covered_medigap,covered_medicaid,covered_chip,covered_military,covered_state,covered_other_gov,covered_single_service,prescription_coverage,uninsured_in_last_year,weight_kg,height_cm,BMI,waist_circumference_cm,hip_circumference_cm,avg_systolic,avg_diastolic,avg_pulse,triglyceride,cholesterol_ldl,cholesterol_hdl,cholesterol_total,insulin,sedentary_minsperday,vigorous_activity_minsperweek,moderate_activity_minsperweek,lifetime_cigarette_smoker,current_cigarette_smoker,ecig_smoker,smokeless_tobacco_user,ratio_income_householdsize
SEQN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
93935,5.9,Prediabetic,male,39,mexican_american,no,other,non_citizen,no_diploma,married or living with partner,4,10000,0.29,uninsured,no,no,no,no,no,no,no,no,no,no_answer,yes,NaN,NaN,NaN,NaN,NaN,108.0,60.3,56.0,148.0,117.0,37.0,184.0,13.40,120.0,0.0,0.0,1.0,2.0,NaN,NaN,2500
93955,7.4,Diabetic,female,66,mexican_american,no,usa,citizen,highschool_grad,divorced/separated,1,15000,0.91,insured,no,yes,no,yes,no,no,no,no,no,yes,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.0,174.0,NaN,180.0,0.0,60.0,1.0,1.0,5.397605e-79,NaN,15000
94542,5.5,Normal,male,30,black,no,usa,citizen,some_college,never married,5,150000,5.00,insured,yes,no,no,no,no,no,no,no,no,yes,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,43.0,89.0,136.0,2.81,720.0,16.0,540.0,2.0,NaN,5.397605e-79,NaN,30000
94724,7.6,Diabetic,female,80,white,no,usa,citizen,highschool_grad,widowed,1,15000,0.99,insured,yes,yes,no,no,no,no,no,no,no,yes,no,NaN,146.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.0,157.0,NaN,720.0,0.0,0.0,2.0,NaN,NaN,NaN,15000
94735,5.8,Prediabetic,female,45,hispanic,no,usa,citizen,highschool_grad,married or living with partner,4,25000,0.82,insured,yes,no,no,no,no,no,no,no,no,yes,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109.0,109.0,57.0,188.0,31.90,180.0,0.0,0.0,2.0,NaN,5.397605e-79,NaN,6250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102391,4.7,Normal,male,31,asian,no,other,citizen,highschool_grad,never married,3,55000,2.17,insured,no,no,no,yes,no,no,no,no,no,yes,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.0,137.0,NaN,480.0,0.0,0.0,2.0,NaN,NaN,NaN,18333
102507,5.4,Normal,male,24,mexican_american,no,usa,citizen,highschool_grad,never married,4,0,NaN,uninsured,no,no,no,no,no,no,no,no,no,no_answer,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122.0,77.0,57.0,158.0,6.25,240.0,3600.0,3600.0,1.0,1.0,5.397605e-79,2.0,0
102590,6.6,Diabetic,male,80,white,yes,usa,citizen,highschool_grad,married or living with partner,2,35000,1.94,insured,no,yes,no,no,no,no,no,no,no,yes,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.0,79.0,51.0,144.0,6.09,30.0,0.0,900.0,1.0,3.0,NaN,NaN,17500


Because many of the 77 rows with missing weights are also missing heights and BMIs, I removed them from the dataset

In [583]:
mega_df = mega_df[~mega_df['weight_kg'].isna()]
mega_df

,glycohemoglobin,calculated_diagnosis,gender,age,race,veteran_status,country_of_birth,citizen_status,education,marital_status,household_size,annual_household_income,income_poverty_ratio,coverage_status,covered_private,covered_medicare,covered_medigap,covered_medicaid,covered_chip,covered_military,covered_state,covered_other_gov,covered_single_service,prescription_coverage,uninsured_in_last_year,weight_kg,height_cm,BMI,waist_circumference_cm,hip_circumference_cm,avg_systolic,avg_diastolic,avg_pulse,triglyceride,cholesterol_ldl,cholesterol_hdl,cholesterol_total,insulin,sedentary_minsperday,vigorous_activity_minsperweek,moderate_activity_minsperweek,lifetime_cigarette_smoker,current_cigarette_smoker,ecig_smoker,smokeless_tobacco_user,ratio_income_householdsize
SEQN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
93705,6.2,Prediabetic,female,66,black,no,usa,citizen,no_diploma,divorced/separated,1,15000,0.82,insured,no,yes,no,yes,no,no,no,no,no,yes,no,79.5,158.3,31.7,101.8,110.0,167.0,66.0,50.7,NaN,NaN,60.0,157.0,NaN,300.0,0.0,120.0,1.0,3.0,NaN,NaN,15000
93706,5.2,Normal,male,18,asian,no,usa,citizen,0,never married,5,0,NaN,insured,yes,no,no,no,no,no,no,no,no,yes,no,66.3,175.7,21.5,79.3,94.4,129.0,71.0,77.3,NaN,NaN,47.0,148.0,NaN,240.0,0.0,285.0,2.0,NaN,NaN,NaN,0
93708,6.2,Prediabetic,female,66,asian,no,other,citizen,no_diploma,married or living with partner,2,35000,1.63,insured,no,yes,no,no,no,no,no,no,no,yes,no,53.5,150.2,23.7,88.2,90.1,146.3,79.7,67.7,58.0,109.0,88.0,209.0,9.72,120.0,0.0,150.0,2.0,NaN,NaN,NaN,17500
93709,6.3,Prediabetic,female,75,black,no,usa,citizen,some_college,widowed,1,10000,0.41,insured,no,yes,no,yes,no,no,no,no,no,yes,no,88.8,151.1,38.9,113.0,131.3,119.0,79.3,62.3,NaN,NaN,65.0,176.0,NaN,600.0,0.0,360.0,1.0,1.0,5.397605e-79,5.397605e-79,10000
93711,5.7,Prediabetic,male,56,asian,no,other,citizen,college_grad,married or living with partner,3,150000,5.00,insured,no,no,no,no,no,no,yes,no,yes,yes,no,62.1,170.6,21.3,86.6,90.7,111.0,62.7,52.0,48.0,156.0,72.0,238.0,5.28,420.0,16.0,360.0,2.0,NaN,NaN,NaN,50000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102949,5.7,Prediabetic,male,33,white,no,usa,citizen,highschool_grad,married or living with partner,5,35000,1.01,insured,no,no,no,yes,no,no,no,no,no,yes,no,76.9,180.1,23.7,85.5,98.0,120.0,76.3,89.7,NaN,NaN,73.0,201.0,NaN,60.0,270.0,2730.0,1.0,1.0,NaN,5.397605e-79,7000
102952,7.4,Diabetic,female,70,asian,no,other,citizen,highschool_grad,married or living with partner,2,20000,0.95,insured,yes,yes,no,no,no,no,no,no,no,yes,no,49.0,156.5,20.0,82.2,87.3,147.7,81.7,82.0,78.0,43.0,60.0,119.0,4.39,120.0,0.0,360.0,2.0,NaN,NaN,NaN,10000
102953,5.9,Prediabetic,male,42,mexican_american,no,other,non_citizen,highschool_grad,divorced/separated,1,15000,NaN,insured,yes,no,no,no,no,no,no,no,no,yes,yes,97.4,164.9,35.8,114.8,112.8,133.3,86.7,77.7,NaN,NaN,49.0,182.0,NaN,360.0,720.0,720.0,1.0,3.0,5.397605e-79,5.397605e-79,15000


In [584]:
mega_df['weight_kg'].isna().sum()

0

In [585]:
mega_df['height_cm'].isna().sum()

9

In [586]:
mega_df[mega_df['height_cm'].isna()]

,glycohemoglobin,calculated_diagnosis,gender,age,race,veteran_status,country_of_birth,citizen_status,education,marital_status,household_size,annual_household_income,income_poverty_ratio,coverage_status,covered_private,covered_medicare,covered_medigap,covered_medicaid,covered_chip,covered_military,covered_state,covered_other_gov,covered_single_service,prescription_coverage,uninsured_in_last_year,weight_kg,height_cm,BMI,waist_circumference_cm,hip_circumference_cm,avg_systolic,avg_diastolic,avg_pulse,triglyceride,cholesterol_ldl,cholesterol_hdl,cholesterol_total,insulin,sedentary_minsperday,vigorous_activity_minsperweek,moderate_activity_minsperweek,lifetime_cigarette_smoker,current_cigarette_smoker,ecig_smoker,smokeless_tobacco_user,ratio_income_householdsize
SEQN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
95651,6.5,Diabetic,male,80,white,yes,usa,citizen,college_grad,married or living with partner,2,55000,3.08,insured,yes,yes,no,no,no,no,no,no,no,yes,no,89.5,NaN,NaN,108.4,105.9,168.3,75.0,60.3,80.0,79.0,56.0,151.0,4.47,300.0,0.0,0.0,2.0,NaN,NaN,NaN,27500
95987,5.5,Normal,male,80,white,no,usa,citizen,college_grad,divorced/separated,1,25000,1.77,insured,no,yes,yes,no,no,no,no,no,no,no,no,79.9,NaN,NaN,NaN,NaN,143.3,78.7,76.3,NaN,NaN,60.0,145.0,NaN,420.0,0.0,2580.0,1.0,3.0,NaN,NaN,25000
96101,5.9,Prediabetic,male,52,asian,no,other,non_citizen,no_diploma,married or living with partner,6,65000,1.90,uninsured,no,no,no,no,no,no,no,no,no,no_answer,yes,79.5,NaN,NaN,97.0,101.8,140.0,87.7,82.3,74.0,104.0,45.0,164.0,7.89,420.0,0.0,60.0,2.0,NaN,NaN,NaN,10833
97693,5.2,Normal,male,80,black,no,usa,citizen,no_diploma,widowed,1,15000,0.99,insured,yes,yes,no,no,no,no,no,no,no,yes,no,61.9,NaN,NaN,81.0,86.7,154.7,73.7,59.7,NaN,NaN,88.0,182.0,NaN,120.0,0.0,0.0,1.0,1.0,NaN,NaN,15000
98414,5.5,Normal,female,70,other,no,usa,citizen,highschool_grad,married or living with partner,2,55000,3.04,insured,yes,yes,no,no,no,no,no,no,no,yes,no,74.1,NaN,NaN,95.3,106.7,106.0,66.3,71.7,NaN,NaN,56.0,145.0,NaN,300.0,1804.0,360.0,2.0,NaN,NaN,NaN,27500
100136,7.8,Diabetic,male,67,white,no,usa,citizen,some_college,married or living with partner,2,0,NaN,insured,no,yes,no,yes,no,no,no,no,yes,yes,yes,112.7,NaN,NaN,NaN,NaN,110.0,58.7,88.0,NaN,NaN,37.0,134.0,NaN,1200.0,0.0,0.0,1.0,3.0,NaN,5.397605e-79,0
100646,5.6,Normal,male,80,white,yes,usa,citizen,highschool_grad,married or living with partner,3,45000,2.11,insured,yes,yes,no,no,no,no,no,no,no,no,no,62.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.0,142.0,NaN,600.0,0.0,0.0,1.0,3.0,NaN,5.397605e-79,15000
100844,5.3,Normal,male,59,black,no,usa,citizen,highschool_grad,divorced/separated,6,150000,3.03,insured,no,yes,no,no,no,no,no,no,no,yes,no,72.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.0,169.0,NaN,360.0,0.0,0.0,2.0,NaN,NaN,NaN,25000
102546,5.7,Prediabetic,male,41,asian,no,other,non_citizen,no_diploma,widowed,3,15000,NaN,insured,no,no,no,yes,no,no,no,no,no,yes,no,68.2,NaN,NaN,95.9,97.4,142.7,92.0,73.7,NaN,NaN,46.0,202.0,NaN,480.0,0.0,1680.0,2.0,NaN,NaN,NaN,5000


In [587]:
mega_df = mega_df[~mega_df['height_cm'].isna()]
mega_df['height_cm'].isna().sum()

0

In [588]:
mega_df['BMI'].isna().sum()

0

In [589]:
mega_df['waist_circumference_cm'].isna().sum()

196

In [590]:
mega_df['hip_circumference_cm'].isna().sum()

185

In [591]:
mega_df['avg_pulse'].isna().sum()

1266

In [592]:
mega_df['avg_pulse'].describe().round(2)

count    3861.00
mean       69.66
std        11.50
min        36.70
25%        61.70
50%        68.70
75%        76.70
max       141.30
Name: avg_pulse, dtype: float64

In [593]:
mega_df['sedentary_minsperday'].describe().round(2)

count    5127.00
mean      384.02
std       751.90
min         0.00
25%       180.00
50%       300.00
75%       480.00
max      9999.00
Name: sedentary_minsperday, dtype: float64

In [594]:
mega_df['sedentary_minsperday'].replace({9999.0: mega_df['sedentary_minsperday'].mean()}, inplace=True)

/Users/kristincooper/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [595]:
mega_df['sedentary_minsperday'].describe().round(2)

count    5127.00
mean      329.64
std       198.46
min         0.00
25%       180.00
50%       300.00
75%       480.00
max      1320.00
Name: sedentary_minsperday, dtype: float64

In [596]:
mega_df['vigorous_activity_minsperweek'].describe().round(2)

count    5127.00
mean      226.78
std       604.79
min         0.00
25%         0.00
50%         0.00
75%        30.00
max      5056.00
Name: vigorous_activity_minsperweek, dtype: float64

In [597]:
mega_df['moderate_activity_minsperweek'].describe().round(2)

count    5127.00
mean      503.54
std       781.30
min         0.00
25%         0.00
50%       180.00
75%       630.00
max      5880.00
Name: moderate_activity_minsperweek, dtype: float64

In [598]:
mega_df.head(1)

,glycohemoglobin,calculated_diagnosis,gender,age,race,veteran_status,country_of_birth,citizen_status,education,marital_status,household_size,annual_household_income,income_poverty_ratio,coverage_status,covered_private,covered_medicare,covered_medigap,covered_medicaid,covered_chip,covered_military,covered_state,covered_other_gov,covered_single_service,prescription_coverage,uninsured_in_last_year,weight_kg,height_cm,BMI,waist_circumference_cm,hip_circumference_cm,avg_systolic,avg_diastolic,avg_pulse,triglyceride,cholesterol_ldl,cholesterol_hdl,cholesterol_total,insulin,sedentary_minsperday,vigorous_activity_minsperweek,moderate_activity_minsperweek,lifetime_cigarette_smoker,current_cigarette_smoker,ecig_smoker,smokeless_tobacco_user,ratio_income_householdsize
SEQN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
93705,6.2,Prediabetic,female,66,black,no,usa,citizen,no_diploma,divorced/separated,1,15000,0.82,insured,no,yes,no,yes,no,no,no,no,no,yes,no,79.5,158.3,31.7,101.8,110.0,167.0,66.0,50.7,NaN,NaN,60.0,157.0,NaN,300.0,0.0,120.0,1.0,3.0,NaN,NaN,15000


In [599]:
preprocessing_vcnulls(mega_df, 'lifetime_cigarette_smoker')

2.0    3038
1.0    2089
Name: lifetime_cigarette_smoker, dtype: int64
Null count: 0


In [600]:
mega_df['lifetime_cigarette_smoker'].replace({1.0: 'yes', 2.0: 'no'}, inplace=True)

/Users/kristincooper/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


# placeholder to git commit

,glycohemoglobin,calculated_diagnosis,gender,age,race,veteran_status,country_of_birth,citizen_status,education,marital_status,household_size,annual_household_income,income_poverty_ratio,coverage_status,covered_private,covered_medicare,covered_medigap,covered_medicaid,covered_chip,covered_military,covered_state,covered_other_gov,covered_single_service,prescription_coverage,uninsured_in_last_year,weight_kg,height_cm,BMI,waist_circumference_cm,hip_circumference_cm,avg_systolic,avg_diastolic,avg_pulse,triglyceride,cholesterol_ldl,cholesterol_hdl,cholesterol_total,insulin,sedentary_minsperday,vigorous_activity_minsperweek,moderate_activity_minsperweek,lifetime_cigarette_smoker,current_cigarette_smoker,ecig_smoker,smokeless_tobacco_user,ratio_income_householdsize
SEQN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


For the first model, I will seek to predict diabetes diagnosis using only easily-accessible features - i.e., questionnaire responses, height, weight, BMI, and blood pressure. 

In [166]:
qx_df = mega_df.drop(columns=['avg_systolic', 'avg_diastolic', 'triglyceride', 'cholesterol_ldl',
                             'cholesterol_hdl', 'cholesterol_total', 'insulin'])
qx_df

,glycohemoglobin,calculated_diagnosis,gender,age,race,veteran_status,country_of_birth,citizen_status,education,marital_status,household_size,annual_household_income,income_poverty_ratio,coverage_status,covered_private,covered_medicare,covered_medigap,covered_medicaid,covered_chip,covered_military,covered_state,covered_other_gov,covered_single_service,prescription_coverage,uninsured_in_last_year,weight_kg,height_cm,BMI,waist_circumference_cm,hip_circumference_cm,avg_pulse,sedentary_minsperday,vigorous_activity_minsperweek,moderate_activity_minsperweek,cigarettes_100+ in lifetime,current_cigarette_smoker,ecig_lastmonth,smokeless_lastmonth,ratio_income_householdsize
SEQN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
93705,6.2,Prediabetic,female,66,black,no,usa,citizen,no_diploma,divorced/separated,1,15000,0.82,insured,no,yes,no,yes,no,no,no,no,no,yes,no,79.5,158.3,31.7,101.8,110.0,50.7,300.0,0.0,120.0,1.0,3.0,NaN,NaN,15000
93706,5.2,Normal,male,18,asian,no,usa,citizen,0,never married,5,0,NaN,insured,yes,no,no,no,no,no,no,no,no,yes,no,66.3,175.7,21.5,79.3,94.4,77.3,240.0,0.0,285.0,2.0,NaN,NaN,NaN,0
93708,6.2,Prediabetic,female,66,asian,no,other,citizen,no_diploma,married or living with partner,2,35000,1.63,insured,no,yes,no,no,no,no,no,no,no,yes,no,53.5,150.2,23.7,88.2,90.1,67.7,120.0,0.0,150.0,2.0,NaN,NaN,NaN,17500
93709,6.3,Prediabetic,female,75,black,no,usa,citizen,some_college,widowed,1,10000,0.41,insured,no,yes,no,yes,no,no,no,no,no,yes,no,88.8,151.1,38.9,113.0,131.3,62.3,600.0,0.0,360.0,1.0,1.0,5.397605e-79,5.397605e-79,10000
93711,5.7,Prediabetic,male,56,asian,no,other,citizen,college_grad,married or living with partner,3,150000,5.00,insured,no,no,no,no,no,no,yes,no,yes,yes,no,62.1,170.6,21.3,86.6,90.7,52.0,420.0,16.0,360.0,2.0,NaN,NaN,NaN,50000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102949,5.7,Prediabetic,male,33,white,no,usa,citizen,highschool_grad,married or living with partner,5,35000,1.01,insured,no,no,no,yes,no,no,no,no,no,yes,no,76.9,180.1,23.7,85.5,98.0,89.7,60.0,270.0,2730.0,1.0,1.0,NaN,5.397605e-79,7000
102952,7.4,Diabetic,female,70,asian,no,other,citizen,highschool_grad,married or living with partner,2,20000,0.95,insured,yes,yes,no,no,no,no,no,no,no,yes,no,49.0,156.5,20.0,82.2,87.3,82.0,120.0,0.0,360.0,2.0,NaN,NaN,NaN,10000
102953,5.9,Prediabetic,male,42,mexican_american,no,other,non_citizen,highschool_grad,divorced/separated,1,15000,NaN,insured,yes,no,no,no,no,no,no,no,no,yes,yes,97.4,164.9,35.8,114.8,112.8,77.7,360.0,720.0,720.0,1.0,3.0,5.397605e-79,5.397605e-79,15000


# Caveats

- This model is not intended to predict diabetes in children under 18 or pregnant women

# Notes to self
- Consider using IterativeImputer to fill education=0
- Consider using IterativeImputer or SimpleImputer to fill annual_household_income=0
- Consider using IterativeImputer to fill income_poverty_ratio nulls 
- Consider using IterativeImputer to fill waist and hip circumference nulls
- Consider using IterativeImputer to fill average pulse nulls

In [167]:
# for ind, row in mega_df.iterrows():
#     if mega_df['age'][ind] < 18:
#         mega_df['education'][ind] = 'child'

I decided to experiment with scikit-learn's experimental IterativeImputer class, which models each feature with missing values as a function of other features, and uses that estimation for imputation. [Read more here](https://scikit-learn.org/stable/modules/impute.html#:~:text=3.-,Multivariate%20feature%20imputation,uses%20that%20estimate%20for%20imputation.&text=Then%2C%20the%20regressor%20is%20used,the%20missing%20values%20of%20y%20.)

In [168]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer